# 0.Libraray Import & Data Load
- 유동인구 데이터 (SKT)
- 카드데이터 (신한카드)
- 유통데이터 (편의점, GS)

- 외부데이터
  - 코로나데이터 (코로나 확진자 수 데이터)
  - 휴일 여부 데이터 (2020, 2019)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 데이터 분석 library
import pandas as pd
import numpy as np

# 통계용 library
import scipy
from scipy.stats.stats import pearsonr # 상관계수
import statsmodels.formula.api as smf # 회귀모델용

!pip install pingouin
import pingouin as pg # 편상관분석용

# 기타
from functools import reduce
import datetime
from datetime import datetime, timedelta
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 225kB 3.2MB/s 
  Created wheel for pingouin: filename=pingouin-0.3.8-cp36-none-any.whl size=221688 sha256=5816acd5b75e253d0de957c9fcfc7f931047bed4d80d1ad2fa87dd6ef8641f0f
  Stored in directory: /root/.cache/pip/wheels/d6/9e/53/f885f73f29cf7c8cac3d8f4b1532bbfef2f5eb543946ac9055
  Created wheel for outdated: filename=outdated-0.2.0-cp36-none-any.whl size=4961 sha256=71bd6847fbd81f4ebffa69e2505a96561b395e988ed0b6504afce80aa7ce3454
  Stored in directory: /root/.cache/pip/wheels/fd/7c/ef/814f514d31197310872b5abf353feb8fef9d67ee658e1e7e39
  Created wheel for littleutils: filename=littleutils-0.2.2-cp36-none-any.whl size=7051 sha256=c345e0f4bbe6186de091113a00c08ee28f8f70dfe5436a3ae66620b93ca09003
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built pingouin outdated littleutils


## 유동인구 데이터 (2020, 2019)
- SKT 유동인구 데이터 2020, 2019 사용

In [ ]:
# 2020년 데이터
skt_age_202002 = pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_202002.CSV", sep="|")
skt_age_202003 = pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_202003.CSV", sep="|")
skt_age_202004 = pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_202004.CSV", sep="|")
skt_age_202005 = pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_202005.CSV", sep="|")

# 2019 유동인구 데이터 age
skt_age_201902=pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_201902.CSV", sep="|")
skt_age_201903=pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_201903.CSV", sep="|")
skt_age_201904=pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_201904.CSV", sep="|")
skt_age_201905=pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_201905.CSV", sep="|")

# skt 데이터 합침 (2020, 2019)
skt_2020 = pd.concat([skt_age_202002,skt_age_202003,skt_age_202004,skt_age_202005]) # 2020년 데이터
skt_2019 = pd.concat([skt_age_201902, skt_age_201903, skt_age_201904, skt_age_201905]) # 2019년 데이터

print(skt_2020.shape, skt_2019.shape)
skt_2020

(8349, 34) (8280, 34)


,STD_YM,STD_YMD,HDONG_CD,HDONG_NM,MAN_FLOW_POP_CNT_0004,MAN_FLOW_POP_CNT_0509,MAN_FLOW_POP_CNT_1014,MAN_FLOW_POP_CNT_1519,MAN_FLOW_POP_CNT_2024,MAN_FLOW_POP_CNT_2529,MAN_FLOW_POP_CNT_3034,MAN_FLOW_POP_CNT_3539,MAN_FLOW_POP_CNT_4044,MAN_FLOW_POP_CNT_4549,MAN_FLOW_POP_CNT_5054,MAN_FLOW_POP_CNT_5559,MAN_FLOW_POP_CNT_6064,MAN_FLOW_POP_CNT_6569,MAN_FLOW_POP_CNT_70U,WMAN_FLOW_POP_CNT_0004,WMAN_FLOW_POP_CNT_0509,WMAN_FLOW_POP_CNT_1014,WMAN_FLOW_POP_CNT_1519,WMAN_FLOW_POP_CNT_2024,WMAN_FLOW_POP_CNT_2529,WMAN_FLOW_POP_CNT_3034,WMAN_FLOW_POP_CNT_3539,WMAN_FLOW_POP_CNT_4044,WMAN_FLOW_POP_CNT_4549,WMAN_FLOW_POP_CNT_5054,WMAN_FLOW_POP_CNT_5559,WMAN_FLOW_POP_CNT_6064,WMAN_FLOW_POP_CNT_6569,WMAN_FLOW_POP_CNT_70U
0,202002,20200201,1114052000,소공동,0.0,88.95,376.68,1077.03,3602.41,6660.39,6519.52,5675.54,5063.35,5992.92,6207.66,5997.45,6356.51,4740.35,5766.20,0.0,73.78,433.73,1408.06,4999.85,6961.02,5115.24,4164.47,3927.93,4058.47,3932.39,3650.70,3361.05,2440.61,2665.43
1,202002,20200202,1114052000,소공동,0.0,51.03,271.96,871.27,2824.97,5031.63,4626.14,3975.63,3455.81,4218.84,4473.19,4183.04,4278.93,3182.24,3567.28,0.0,55.15,327.72,1231.75,3807.14,4702.91,3698.04,2922.33,2781.63,2947.01,2632.34,2669.56,2399.90,1704.22,2026.77
2,202002,20200203,1114052000,소공동,0.0,32.47,136.94,826.26,3580.23,8823.11,11803.45,14934.31,14823.99,16430.05,14201.85,10908.99,9628.67,6658.04,7411.35,0.0,35.09,155.99,1166.65,6301.35,11109.59,10754.75,9655.01,9018.58,8786.59,7257.35,5710.55,4935.00,3070.67,3349.99
3,202002,20200204,1114052000,소공동,0.0,27.34,154.75,823.69,3510.75,9361.85,12127.55,15650.18,15484.99,16854.01,14653.71,11550.17,9567.69,6884.62,7319.09,0.0,40.41,193.20,1148.80,6345.14,11771.06,11467.86,9834.22,9323.74,8982.67,7448.09,5938.57,4948.10,3091.30,3494.91
4,202002,20200205,1114052000,소공동,0.0,26.30,177.28,809.21,3579.35,9631.85,12378.43,15876.88,15624.94,16730.33,14462.22,11456.97,9360.22,6709.08,6951.99,0.0,36.44,172.20,1285.73,6587.78,11876.40,11285.83,9972.17,9299.52,8976.55,7183.79,5719.89,4822.39,2960.73,3204.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,202005,20200527,2726069000,고산3동,0.0,70.07,519.05,956.45,1205.44,1507.57,1500.51,2014.87,2295.21,2947.78,3020.17,2653.97,2217.68,1561.13,1492.80,0.0,114.34,552.87,842.82,1264.78,1201.14,1106.57,1728.34,2209.31,2808.54,2721.54,2070.62,1613.50,1086.34,1143.37
2135,202005,20200528,2726069000,고산3동,0.0,99.04,749.36,1085.68,1639.82,1777.32,1776.02,2137.50,2582.56,3415.30,3458.23,3019.66,2550.45,1845.30,1780.62,0.0,146.62,733.55,1117.85,1597.98,1377.94,1258.02,1847.61,2677.19,3342.68,3036.71,2389.70,1848.92,1291.19,1515.36
2136,202005,20200529,2726069000,고산3동,0.0,155.70,781.70,1235.94,1706.97,1982.74,1924.60,2500.02,2810.11,3625.34,3718.35,3212.18,2852.53,1755.48,1811.35,0.0,111.30,801.94,1167.51,1709.27,1583.58,1487.06,2149.30,2916.87,3643.31,3395.24,2800.02,2043.25,1356.15,1455.89
2137,202005,20200530,2726069000,고산3동,0.0,119.89,903.85,1236.60,1722.01,2142.41,2106.49,2215.86,2649.30,3713.62,3854.18,3138.66,2506.90,1683.99,1666.55,0.0,152.58,727.46,1229.17,1627.58,1644.73,1420.31,1812.27,2640.71,3362.25,2909.34,2353.82,1812.56,1242.56,1372.38


In [ ]:
# skt 유동인구 데이터

# 2020 유동인구 데이터 연령 총합 변수 만들기
skt_2020['sum_20'] = skt_2020.iloc[:,4:].sum(axis=1)
skt_2020 = skt_2020[['STD_YMD','HDONG_NM','sum_20']]
skt_2020.columns = ['STD_DD','HDONG_NM','sum_20']
skt_2020 = skt_2020.reset_index(drop=True)

# 2019 유동인구 데이터 연령 총합 변수 만들기
skt_2019["sum_19"] = skt_2019.iloc[:,4:].sum(axis=1)
skt_2019 = skt_2019[["STD_YMD","HDONG_NM","sum_19"]]
skt_2019.columns = ['STD_DD','HDONG_NM','sum_19']
skt_2019 = skt_2019.reset_index(drop=True)

print(skt_2019.shape, skt_2019.shape)
skt_2019

(8280, 3) (8280, 3)


,STD_DD,HDONG_NM,sum_19
0,20190201,소공동,242230.68
1,20190202,소공동,133411.99
2,20190203,소공동,90124.93
3,20190204,소공동,83378.55
4,20190205,소공동,111009.47
...,...,...,...
8275,20190527,고산3동,47011.48
8276,20190528,고산3동,49344.74
8277,20190529,고산3동,50345.41
8278,20190530,고산3동,50148.22


## 카드데이터 (2020, 2019)
- 신한카드 데이터 (외국인 + 내국인)
- 신한카드 행정동 데이터
- 카드 데이터 + 행정동 데이터 Merge
- 2020, 2019 데이터 분리
- 카드데이터 업종별 대분류 리스트 변수 생성

In [ ]:
# 신한 카드 데이터 (내국인, 외국인)
card_re = pd.read_csv('/content/drive/My Drive/big_contest/shinhan/CARD_SPENDING_RESIDENT.txt', sep = "\t", engine='python', encoding = "cp949")
card_fo = pd.read_csv('/content/drive/My Drive/big_contest/shinhan/CARD_SPENDING_FOREIGNER.txt', sep = "\t", engine='python', encoding = "cp949")

# 신한 카드 행정동 데이터
dong = pd.read_excel('/content/drive/My Drive/big_contest/shinhan/02_혁신아이디어분야_카드소비데이터(신한카드)_데이터정의서.xlsx', sheet_name='(참고) 행정동코드',index_col=0, header=None)

print(card_re.shape, card_fo.shape, dong.shape)

(2537753, 8) (238829, 7) (71, 4)


In [ ]:
# 카드 행정동 데이터 전처리
dong.columns = ['GU_CD','DONG_CD','GU_NM','HDONG_NM']
dong.reset_index(inplace=True, drop=True)
dong = dong.loc[2:]
dong = dong.reset_index(drop=True)

# 컬럼 type int 형으로 변경
dong['GU_CD'] = dong['GU_CD'].astype(int)
dong['DONG_CD'] = dong['DONG_CD'].astype(int)

# 카드 데이터 (내국인 + 외국인) concat
card = pd.concat([card_fo,card_re])

# 행정동 이름과 카드 데이터 merge
card = pd.merge(card, dong, on=['GU_CD','DONG_CD'])

print(card.shape)
card

(2776582, 11)


,STD_DD,GU_CD,DONG_CD,MCT_CAT_CD,COUNTRY_NM,USE_CNT,USE_AMT,SEX_CD,AGE_CD,GU_NM,HDONG_NM
0,20190204,260,510,81,대만,2,30.0,NaN,NaN,대구 수성구,범어1동
1,20190213,260,510,40,대만,2,32.0,NaN,NaN,대구 수성구,범어1동
2,20190416,260,510,40,대만,2,16.0,NaN,NaN,대구 수성구,범어1동
3,20190419,260,510,42,대만,3,610.0,NaN,NaN,대구 수성구,범어1동
4,20190308,260,510,10,대만,7,629.0,NaN,NaN,대구 수성구,범어1동
...,...,...,...,...,...,...,...,...,...,...,...
2776577,20200531,350,710,81,NaN,47,570.0,M,45.0,서울 노원구,상계9동
2776578,20200531,350,710,81,NaN,33,444.0,M,50.0,서울 노원구,상계9동
2776579,20200531,350,710,81,NaN,28,403.0,M,55.0,서울 노원구,상계9동
2776580,20200531,350,710,81,NaN,33,384.0,M,60.0,서울 노원구,상계9동


In [ ]:
# 일별, 구별 ,동별 USE_CNT, USE_AMT 합
card_group = card.groupby(["STD_DD",'GU_NM','HDONG_NM'], as_index=False)['USE_CNT','USE_AMT'].agg("sum")

# 2020, 2019 카드 데이터 분리
card_2020 = card_group[card_group.STD_DD>=20200101] # 2020년 카드 데이터
card_2019 = card_group[card_group.STD_DD<20200101] # 2019년 카드 데이터

print(card_2020.shape, card_2019.shape)

(8349, 5) (8280, 5)


In [ ]:
# 카드 대분류 list dictionary
type_list = {'10':'숙박(10)', '20':'레저용품(20)', '21':'레저업소(21)', 
            '22':'문화취미(22)', '30':'가구(30)' , '31':'전기(31)' ,
            '32':'주방용구(32)', '33':'연료판매(33)' , '34':'광학제품(34)',
            '35':'가전(35)', '40':'유통업(40)', '42':'의복(42)', 
            '43':'직물(43)', '44':'신변잡화(44)', '50':'서적문구(50)',
            '52':'사무통신(52)', '60':'자동차판매(60)','62':'자동차정비(62)',
            '70':'의료기관(70)', '71':'보건위생(71)', '80':'요식업소(80)',
            '81':'음료식품(81)', '92':'수리서비스(92)'}

# 업종 list
MCT_CAT_CD_list = card['MCT_CAT_CD'].unique()
MCT_CAT_CD_list = sorted(MCT_CAT_CD_list)
MCT_CAT_CD_list = list(map(str, MCT_CAT_CD_list))
print(len(MCT_CAT_CD_list), MCT_CAT_CD_list)

23 ['10', '20', '21', '22', '30', '31', '32', '33', '34', '35', '40', '42', '43', '44', '50', '52', '60', '62', '70', '71', '80', '81', '92']


## 편의점(유통, GS) 데이터 (2020,2019)
- GS 제공 편의점 데이터
- 2020, 2019 데이터 분리

In [ ]:
# GS 데이터 가져오기
gs = pd.read_excel('/content/drive/My Drive/big_contest/GS/유통(GS리테일).xlsx', sheet_name='종합테이블(TB_DD_STR_PURCH_CLS)')
print(gs.shape)
gs

(14460, 13)


,OPER_DT,PVN,BOR,ADMD,AMT_IND,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P
0,20190201,대구광역시,수성구,고산1동,1.064336,0.450848,0.172221,0.295458,0.023263,0.044679,0.005075,0.004820,0.003637
1,20190201,대구광역시,수성구,고산2동,1.076418,0.457937,0.196274,0.295564,0.005521,0.034056,0.000844,0.008893,0.000913
2,20190201,대구광역시,수성구,고산3동,0.856071,0.359978,0.196602,0.376762,0.013408,0.029606,0.009117,0.008831,0.005697
3,20190201,대구광역시,수성구,두산동,0.724868,0.281969,0.175207,0.363202,0.024235,0.134191,0.000000,0.021196,0.000000
4,20190201,대구광역시,수성구,만촌1동,1.155728,0.400983,0.200454,0.310531,0.038868,0.035427,0.005284,0.005734,0.002719
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14455,20200531,서울특별시,중구,중림동,1.595038,0.314217,0.221087,0.344307,0.026204,0.070094,0.006347,0.013541,0.004203
14456,20200531,서울특별시,중구,청구동,1.389184,0.276333,0.221216,0.363681,0.053907,0.076963,0.001592,0.000353,0.005955
14457,20200531,서울특별시,중구,필동,0.714265,0.303338,0.187615,0.370371,0.038908,0.079408,0.007826,0.008298,0.004236
14458,20200531,서울특별시,중구,황학동,1.134190,0.264708,0.257387,0.367001,0.029140,0.070770,0.001060,0.003646,0.006287


In [ ]:
# 차후 군집별 업종 상관관계, 회귀 모델용 데이터 변수
gs_row = gs.copy()

# GS 데이터 필요 column 줄이기 (구조방정식, 매개효과 용 변수)
gs.columns=['STD_DD', 'HGU_NM', 'CTGG_NM', 'HDONG_NM', 'AMT_IND', 'LCLS_10_P', 'LCLS_20_P',
            'LCLS_30_P', 'LCLS_40_P', 'LCLS_50_P', 'LCLS_60_P', 'LCLS_70_P', 'LCLS_80_P']
gs = gs[['STD_DD', 'HGU_NM', 'HDONG_NM', 'AMT_IND']]
gs

,STD_DD,HGU_NM,HDONG_NM,AMT_IND
0,20190201,대구광역시,고산1동,1.064336
1,20190201,대구광역시,고산2동,1.076418
2,20190201,대구광역시,고산3동,0.856071
3,20190201,대구광역시,두산동,0.724868
4,20190201,대구광역시,만촌1동,1.155728
...,...,...,...,...
14455,20200531,서울특별시,중림동,1.595038
14456,20200531,서울특별시,청구동,1.389184
14457,20200531,서울특별시,필동,0.714265
14458,20200531,서울특별시,황학동,1.134190


In [ ]:
# 유통 GS 품목별 대분류 list dictionary
circulation_list = {'mul_LCLS_10_P':'식사',
                    'mul_LCLS_20_P':'간식',
                    'mul_LCLS_30_P':'마실거리',
                    'mul_LCLS_40_P':'홈&리빙',
                    'mul_LCLS_50_P':'헬스&뷰티',
                    'mul_LCLS_60_P':'취미&여가활동_비중',
                    'mul_LCLS_70_P':'사회활동_비중',
                    'mul_LCLS_80_P':'임신/육아_비중',
                    'mul_LCLS_90_P':'기호품_비중'}

# 유통 GS 품목별 대분류 list dictionary
circulation_list_reverse = {'식사':               'mul_LCLS_10_P',
                            '간식':               'mul_LCLS_20_P',
                            '마실거리' :          'mul_LCLS_30_P',
                            '홈&리빙':            'mul_LCLS_40_P',
                            '헬스&뷰티':          'mul_LCLS_50_P',
                            '취미&여가활동_비중': 'mul_LCLS_60_P',
                            '사회활동_비중':      'mul_LCLS_70_P',
                            '임신/육아_비중':     'mul_LCLS_80_P',
                            '기호품_비중':        'mul_LCLS_90_P'}

## 코로나 데이터 (2020)
- 출처 : https://github.com/jooeungen/coronaboard_kr // CoronaBoard_kr 깃허브 
- 출처 : http://ncov.mohw.go.kr/ // corona board에서 참조한 질병관리본부 링크
- <b>CoronaBoard data - Available Commercial & Non-commercial</b>
  - kr_daily.csv (코로나 일일 확진자 수 - 누적) (원본)
  - kr_regional_data.csv (코로나 지역별  일일 확진자 수 - 누적 )(원본)

- <b>사용데이터 : 코로나 일일 확진자 수 데이터 (전국, 서울, 대구)</b>
  - <b>위 출처 데이터를 1차 가공(차분)한 데이터 (누적 -> 일일) </b>
  - 기간 : 20200201 ~ 20200601 데이터 사용
  - covid_all.csv : 전국 코로나 일일 확진자 수
  - corona_seoul.csv : 서울지역 코로나 일일 확진자 수
  - corona_daegu.csv : 대구지역 코로나 일일 확진자 수
  - 코로나 데이터는 하루씩 당겨서 사용
  

In [ ]:
# 코로나 전국 확진자 데이터
covid = pd.read_csv("/content/drive/My Drive/big_contest/corona/covid_all.csv", index_col=0, encoding="cp949")

# 서울, 대구 각각의 지역 확진자 데이터
covid_seoul = pd.read_csv("/content/drive/My Drive/big_contest/corona/corona_seoul.csv", index_col=0, encoding="cp949")
covid_daegu = pd.read_csv("/content/drive/My Drive/big_contest/corona/corona_daegu.csv", index_col=0, encoding="cp949")

# 전국 코로나 데이터
covid = covid.reset_index()
covid.columns = ['STD_DD','confirmed']
covid

,STD_DD,confirmed
0,20200201,4
1,20200202,1
2,20200203,3
3,20200204,0
4,20200205,1
...,...,...
179,20200729,28
180,20200730,48
181,20200731,18
182,20200801,36


In [ ]:
# corona 지역 데이터 (서울, 대구)
covid_seoul = covid_seoul[['date','confirmed_diff']]
covid_seoul.columns=['STD_DD','seoul_confirmed']
covid_daegu = covid_daegu[['date','confirmed_diff']]
covid_daegu.columns=['STD_DD','daegu_confirmed']

# 서울, 대구 코로나 데이터
print(covid_seoul.shape, covid_daegu.shape)
covid_seoul.head(3)

(121, 2) (121, 2)


,STD_DD,seoul_confirmed
1,20200201,0
2,20200202,1
3,20200203,0


## 휴일여부 데이터 (2019, 2020)
- 2019년, 2020년 휴일 여부 데이터
- 직접 데이터 수작업 작성
- 연휴 명절, 대체휴일 등 반영

In [ ]:
# 2019, 2020 휴일 여부 데이터
holiday = pd.read_csv("/content/drive/My Drive/big_contest/holiday/holiday.csv")

print(holiday.shape)
holiday

(241, 3)


,date,day,weekend
0,20190201,4,0
1,20190202,5,1
2,20190203,6,1
3,20190204,0,1
4,20190205,1,1
...,...,...,...
236,20200527,2,0
237,20200528,3,0
238,20200529,4,0
239,20200530,5,1


In [ ]:
# 컬럼명 변경, type 변경
holiday.columns = ['STD_DD','day','weekend']

holiday['STD_DD'] = holiday['STD_DD'].astype(str)
holiday['STD_DD'] = holiday['STD_DD'].str.replace("-","")
holiday['STD_DD'] = holiday['STD_DD'].astype(int)
holiday

,STD_DD,day,weekend
0,20190201,4,0
1,20190202,5,1
2,20190203,6,1
3,20190204,0,1
4,20190205,1,1
...,...,...,...
236,20200527,2,0
237,20200528,3,0
238,20200529,4,0
239,20200530,5,1


## 서울 대구 행정동 리스트
- 서울 중구/노원구 행정동 리스트
- 대구 중구/수성구 행정동 리스트

In [ ]:
# 분석의 편의를 위해 서울시 중구, 서울시 노원구, 대구시 중구, 대구시 수성구 동별로 리스트 생성
seoul_jung_list = ['소공동', '회현동', '명동', '필동', '장충동', '광희동', '을지로동', '신당동', '다산동','약수동', '청구동', '신당5동', '동화동', '황학동', '중림동']
seoul_nowon_list = ['월계1동', '월계2동', '월계3동','공릉1동', '공릉2동', '하계1동', '하계2동', '중계본동', '중계1동', '중계4동', '중계2.3동','상계1동', '상계2동', '상계3.4동', '상계5동', '상계6.7동', '상계8동', '상계9동','상계10동']
daegu_jung_list = ['동인동', '삼덕동', '성내1동', '성내2동', '성내3동', '대신동', '남산1동', '남산2동', '남산3동', '남산4동', '대봉1동', '대봉2동']
daegu_suseong_list = ['범어1동', '범어2동', '범어3동', '범어4동', '만촌1동', '만촌2동', '만촌3동', '수성1가동', '수성2.3가동', '수성4가동', '황금1동', 
                      '황금2동', '중동', '상동', '파동', '두산동', '지산1동', '지산2동', '범물1동', '범물2동', '고산1동', '고산2동', '고산3동']

# 3.매개효과 & 부트스트래핑 검정
- 신규 코로나 확진자(𝑿)와 오프라인 매출액(𝐘)의 관계에서 2020 유동인구(𝐌)를 매개변인으로 한 매개모델을 가정

- 종속변수를 2020 유동인구(𝑴), 독립변수를 신규 코로나 확진자(𝑿)로 한 단일회귀 (𝐌= 𝜷_𝟎+𝒂∙𝑿) 의 비표준화 회귀계수(𝒂)를 구함

- 종속변수를 2020 카드/GS 매출액(𝐘), 독립변수를 신규 코로나 확진자(𝑿), 2020 유동인구(𝑴)인 다중 회귀(𝐘= 𝜷_𝟎+𝒃∙𝑿+𝒄∙𝑴)에서의 비표준화 회귀계수(𝒃, 𝒄) 를 구함

## 부트스트래핑 검정 소스코드
- human_learning_3_1_부트스트랩핑카드평일.ipynb
- human_learning_3_2_부트스트랩핑카드휴일.ipynb
- human_learning_3_3_부트스트랩핑편의점평일.ipynb
- human_learning_3_4_부트스트랩핑편의점휴일.ipynb

- 위 4개의 ipynb에서 얻은 파일들
  - 카드평일부트스트래핑_신뢰95_비표준화_10000.csv
  - 카드휴일부트스트래핑_신뢰95_비표준화_10000.csv
  - 편의점평일부트스트래핑_신뢰95_비표준화_10000.csv
  - 편의점휴일부트스트래핑_신뢰95_비표준화_10000.csv

## 순수 매개효과 조건 1,2,3
- human_learning_3_0_부트스트랩핑카드평일.ipynb 에서 위 4개의 ipynb파일 결과를 종합 받고 조건1,2,3 에 대해서 진행 -> 최종 순수매개효과 지역 추출

### 조건1
- 간접효과 𝒂⦁𝒄는 통계적 유의성을 위해 Bootstrapping 검정을 진행하며, 직접효과 𝒃는 p-value 검정을 통해 유의 여부를 판단한다 

- 회귀계수 𝒂, 𝒄 각각은 t분포를 가정하고 통계적 유의성을 검정
- 𝒂⦁𝒄 는 회귀계수 𝒂 와 𝒄 의 Cross Product 이기 때문에 분포가 불분명
- 따라서 Bootstrapping 기법을 사용해 𝒂⦁𝒄 의 표본 분포를 추정하고 신뢰구간으로 통계적 유의성을 검증

- 따라서 해당 데이터를 가지고 10000번의 30% random sampling을 통해 모집단을 대표할 수 있는 분포를 생성

- 그 후 95% 신뢰구간에서 0이 포함된 매개효과 통계량을 통계적으로 유의하지 않다고 판단

### 조건2 
- 직접효과(b)는 유의하지 않고, 간접효과(ac)만 유의한 경우 분석 대상에 포함
- 직접효과 (b)가 유의시 해당 지역은 기각

### 조건3
- a가 음수 c가 양수인경우만 선택 
- 신규 코로나 확진자 증가로인해 유동인구가 감소하였고, 유동인구 감소로 인해 매출도 같이 감소한 경우

### 최종 결과 순수 매개효과, Bootstraping, 조건1,2,3 결과 
  - boot_direct_mediation_result.csv



## 3-1. 기존 군집결과 정리

In [ ]:
# 서울, 대구 군집 리스트
# 1군집 (상권중심지역) /  2군집 (주거상권지역) / 3군집 (주거중심지역) / 4군집 (교통중심지역)

# 서울 cluster 1,2,3,4
seoul_cluster1 = ['명동', '상계2동', '소공동', '필동', '회현동']
seoul_cluster2 = ['동화동', '상계1동', '상계9동', '월계1동', '중계1동', '중계2.3동', '중계4동', '중계본동', '중림동', '하계1동']
seoul_cluster3 = ['상계10동', '상계3.4동', '상계5동', '상계8동', '월계2동', '월계3동', '청구동', '하계2동']
seoul_cluster4 = ['공릉1동', '공릉2동', '광희동', '다산동', '상계6.7동', '신당5동', '신당동', '약수동', '을지로동', '장충동', '황학동']

# 대구 cluster 1,2,3,4
daegu_cluster1 = ['대봉1동', '대신동', '두산동', '범어1동', '범어2동', '삼덕동', '성내1동', '성내2동', '중동']
daegu_cluster2 = ['고산1동', '고산3동', '남산4동', '만촌3동', '범물1동', '범어3동', '범어4동', '지산1동', '지산2동']
daegu_cluster3 = ['남산3동', '대봉2동', '범물2동', '상동', '수성1가동', '수성4가동', '파동', '황금1동']
daegu_cluster4 = ['고산2동', '남산1동', '남산2동', '동인동', '만촌1동', '만촌2동', '성내3동', '수성2.3가동', '황금2동']

# 서울 34개 동, 대구 35개 동
print(len(seoul_cluster1) + len(seoul_cluster2) + len(seoul_cluster3) + len(seoul_cluster4))
print(len(daegu_cluster1) + len(daegu_cluster2) + len(daegu_cluster3) + len(daegu_cluster4))

# 서울 , 대구 구분 없는 군집 리스트
cluster_1=['명동' ,'상계2동', '소공동' ,'필동', '회현동','대봉1동' ,'대신동', '두산동' ,'범어1동' ,'범어2동' ,'삼덕동', '성내1동', '성내2동' ,'중동']
cluster_2=['동화동' ,'상계1동', '상계9동' ,'월계1동', '중계1동' ,'중계2.3동', '중계4동' ,'중계본동', '중림동' ,'하계1동','고산1동' ,'고산3동' ,'남산4동', '만촌3동', '범물1동', '범어3동' ,'범어4동', '지산1동', '지산2동']
cluster_3=['상계10동', '상계3.4동' ,'상계5동', '상계8동' ,'월계2동' ,'월계3동' ,'청구동' ,'하계2동','남산3동' ,'대봉2동' ,'범물2동', '상동', '수성1가동' ,'수성4가동' ,'파동' ,'황금1동']
cluster_4=['공릉1동', '공릉2동' ,'광희동', '다산동', '상계6.7동', '신당5동', '신당동', '약수동', '을지로동' ,'장충동', '황학동','고산2동', '남산1동', '남산2동', '동인동', '만촌1동', '만촌2동', '성내3동', '수성2.3가동', '황금2동']
print(len(cluster_1), len(cluster_2), len(cluster_3), len(cluster_4))

34
35
14 19 16 20


## 3-2. 매개효과,부트스트래핑 검정결과

In [ ]:
cor_val = 0.35 # 0.3 # 0.35

# 조건 1,2,3

# 부트스트랩 & 직접효과 제거 결과 read 해보기

# 부트스트랩 & 직접효과 제거 결과
boot_direct_mediation_result = pd.read_csv('/content/drive/My Drive/big_contest/bootstraping/boot_direct_mediation_result.csv', encoding='cp949')

# 부트스트랩 & 직접효과 제거 결과
card_weekday_list = boot_direct_mediation_result.iloc[0, ].values[0].split(',')
card_weekend_list = boot_direct_mediation_result.iloc[1, ].values[0].split(',')
gs_weekday_list = boot_direct_mediation_result.iloc[2, ].values[0].split(',')
gs_weekend_list = boot_direct_mediation_result.iloc[3, ].values[0].split(',')

print('카드 평일 부트스트랩, 직접효과,  제거 : ', len(card_weekday_list), card_weekday_list)
print('카드 휴일 부트스트랩, 직접효과 제거 : ', len(card_weekend_list), card_weekend_list)
print('편의점 평일 부트스트랩, 직접효과 제거 : ', len(gs_weekday_list), gs_weekday_list)
print('편의점 휴일 부트스트랩, 직접효과 제거 : ', len(gs_weekend_list), gs_weekend_list)

카드 평일 부트스트랩, 직접효과,  제거 :  44 ['범물1동', '상계1동', '중계4동', '상계5동', '공릉2동', '명동', '하계2동', '삼덕동', '대봉2동', '만촌3동', '만촌2동', '두산동', '을지로동', '장충동', '성내2동', '만촌1동', '회현동', '소공동', '월계1동', '범어3동', '범어1동', '수성4가동', '하계1동', '범어2동', '남산3동', '상계10동', '상동', '범어4동', '남산2동', '남산1동', '수성2.3가동', '성내1동', '청구동', '대봉1동', '신당5동', '남산4동', '황금2동', '월계2동', '중계1동', '중림동', '공릉1동', '대신동', '중계2.3동', '수성1가동']
카드 휴일 부트스트랩, 직접효과 제거 :  45 ['범물1동', '상계1동', '중계4동', '상계5동', '공릉2동', '명동', '성내3동', '대봉2동', '만촌3동', '만촌2동', '고산3동', '두산동', '을지로동', '중동', '동인동', '성내2동', '만촌1동', '약수동', '장충동', '소공동', '상계6.7동', '범어1동', '하계1동', '범어2동', '황금1동', '상계10동', '필동', '상동', '범어4동', '남산2동', '남산1동', '파동', '중계본동', '고산1동', '고산2동', '대봉1동', '신당5동', '남산4동', '황금2동', '월계2동', '상계2동', '신당동', '중림동', '대신동', '중계2.3동']
편의점 평일 부트스트랩, 직접효과 제거 :  45 ['범물1동', '상계1동', '상계5동', '공릉2동', '월계3동', '황학동', '성내3동', '수성1가동', '만촌3동', '상계9동', '고산3동', '상계3.4동', '두산동', '을지로동', '중동', '동인동', '장충동', '약수동', '회현동', '소공동', '월계1동', '범어3동', '상계6.7동', '범어1동', '수성4가동', '하계1동', '범어2동', '황금1동

In [ ]:
# 서울, 대구 분리
print('카드 평일 서울', len(set(seoul_nowon_list+seoul_jung_list).intersection(set(card_weekday_list))))
print('카드 휴일 서울', len(set(seoul_nowon_list+seoul_jung_list).intersection(set(card_weekend_list))))
print('카드 평일 대구', len(set(daegu_jung_list+daegu_suseong_list).intersection(set(card_weekday_list))))
print('카드 휴일 대구', len(set(daegu_jung_list+daegu_suseong_list).intersection(set(card_weekend_list))))

print('편의점 평일 서울', len(set(seoul_nowon_list+seoul_jung_list).intersection(set(gs_weekday_list))))
print('편의점 휴일 서울', len(set(seoul_nowon_list+seoul_jung_list).intersection(set(gs_weekend_list))))
print('편의점 평일 대구', len(set(daegu_jung_list+daegu_suseong_list).intersection(set(gs_weekday_list))))
print('편의점 휴일 대구', len(set(daegu_jung_list+daegu_suseong_list).intersection(set(gs_weekend_list))))


카드 평일 서울 20
카드 휴일 서울 20
카드 평일 대구 24
카드 휴일 대구 25
편의점 평일 서울 24
편의점 휴일 서울 29
편의점 평일 대구 21
편의점 휴일 대구 23


# 4.서울, 대구 각 군집별 상관비교분석
- 'HumanLearning_1.EDA&Clustering&Visualization' 에서 나눈 군집을 사용
- 'HumanLearning_3_0_매개효과&부트스트래핑_결과종합 검정 최종 통과 지역 사용

- Problem : 신규 코로나 확진자와 2019년도 매출액의 상관관계가 높고 2020년도 매출액이 2019년도 매출액과 비슷하게 움직였기 때문에 해당 업종이 코로나에 민감한 업종으로 보일 수 있다.

- Solution : 상관계수와 부분상관계수 비교를 통한 업종/품목 우연성 제거
  - Pearson’s Correlation 과 Semi-partial Correlation 비교를 통해 해당 관계에 우연성이 있는 업종을 제거
  - (Semi-partial correlation은 19년도 매출액 데이터로 신규 코로나 확진자 수를 통제 한 후 20년도와 19년도 매출액 상관을 구한 것)

### 순수매개효과 조건1,2,3 통과지역
- 카드 평일 순수매개효과 조건1,2,3 
- 카드 휴일 순수매개효과 조건1,2,3 
- 편의점 평일 순수매개효과 조건1,2,3 
- 편의점 휴일 순수매개효과 조건1,2,3 

### 서울 / 대구
- 1군집 (상권지역)
- 2군집 (주거상권지역)
- 3군집 (주거지역)
- 4군집 (교통중심지역)

### ==> Pearson's과 Semi-partial Correlation 비교를 통한 업종별 상관분석 진행 및 우연성 제거

## 4-1. 군집별 카드 업종별 상관분석
- 서울 평일 / 서울 휴일 / 대구 평일 / 대구 휴일

### 데이터 Cleansing & 전처리
- 카드데이터 2020 구분
  - card2020
- 카드 휴일, 휴일 데이터 구분
  - card2020_weekday, card2020_weekend

In [ ]:
# card2020 년도 데이터
card2020 = card[card['STD_DD']>20191231]
card2020 = pd.merge(card2020, holiday, on='STD_DD')

# 평일, 휴일 나누기
card2020_weekday = card2020[card2020['weekend']==0]
card2020_weekend = card2020[card2020['weekend']==1]

print(card2020.shape, card2020_weekday.shape, card2020_weekend.shape)

# card2019 년도 데이터 
card2019 = card[card['STD_DD']<20191231]
card2019 = pd.merge(card2019, holiday, on='STD_DD')

# 2019년과 2020년을 날짜별(같은 주차의 같은 요일)로 맞추어주는 과정
card2019['STD_DD'] = card2019['STD_DD'].astype('str')
card2019['STD_DD'] = pd.to_datetime(card2019['STD_DD']) + timedelta(weeks=52)

# 2019년을 2020 년도 로 다 바꾸기
card2019['STD_DD'] = card2019['STD_DD'].astype(str)
card2019["STD_DD"] = card2019["STD_DD"].str.replace("-","")
card2019["STD_DD"] = card2019["STD_DD"].astype(int)

# 평일, 휴일 나누기
card2019_weekday = card2019[card2019['weekend']==0]
card2019_weekend = card2019[card2019['weekend']==1]

print(card2019.shape, card2019_weekday.shape, card2019_weekend.shape)

(1310849, 13) (906029, 13) (404820, 13)
(1465733, 13) (1008488, 13) (457245, 13)


### 카드 서울 평일 (2020, 2019)


In [ ]:
# 2020년도 데이터
# 각 cluster와 서울 카드 평일 교집합 
seoul_weekday_seg1_list = list(set(seoul_cluster1).intersection(set(card_weekday_list)))
seoul_weekday_seg2_list = list(set(seoul_cluster2).intersection(set(card_weekday_list)))
seoul_weekday_seg3_list = list(set(seoul_cluster3).intersection(set(card_weekday_list)))
seoul_weekday_seg4_list = list(set(seoul_cluster4).intersection(set(card_weekday_list)))
print(seoul_weekday_seg1_list, '\n', seoul_weekday_seg2_list, '\n', seoul_weekday_seg3_list, '\n', seoul_weekday_seg4_list)

# 서울 평일 각 seg 로 분리 데이터프레임 생성
card_seoul_weekday_seg1 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(seoul_weekday_seg1_list)]
card_seoul_weekday_seg2 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(seoul_weekday_seg2_list)] 
card_seoul_weekday_seg3 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(seoul_weekday_seg3_list)]
card_seoul_weekday_seg4 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(seoul_weekday_seg4_list)]

print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)

# 평일 각 seg별 일별, 업종별 USE_AMT 평균
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg1 = pd.DataFrame(card_seoul_weekday_seg1)

card_seoul_weekday_seg2 = card_seoul_weekday_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg2 = pd.DataFrame(card_seoul_weekday_seg2)

card_seoul_weekday_seg3 = card_seoul_weekday_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg3 = pd.DataFrame(card_seoul_weekday_seg3)

card_seoul_weekday_seg4 = card_seoul_weekday_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg4 = pd.DataFrame(card_seoul_weekday_seg4)

print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)

# 데이터 reset_index
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.reset_index()
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.reset_index()
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.reset_index()
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_seoul_weekday_seg1["MCT_CAT_CD"] = card_seoul_weekday_seg1["MCT_CAT_CD"].astype(str)
card_seoul_weekday_seg2["MCT_CAT_CD"] = card_seoul_weekday_seg2["MCT_CAT_CD"].astype(str)
card_seoul_weekday_seg3["MCT_CAT_CD"] = card_seoul_weekday_seg3["MCT_CAT_CD"].astype(str)
card_seoul_weekday_seg4["MCT_CAT_CD"] = card_seoul_weekday_seg4["MCT_CAT_CD"].astype(str)
print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)

['명동', '소공동', '회현동'] 
 ['하계1동', '중계1동', '중계4동', '중림동', '월계1동', '중계2.3동', '상계1동'] 
 ['청구동', '상계5동', '하계2동', '상계10동', '월계2동'] 
 ['공릉2동', '공릉1동', '신당5동', '을지로동', '장충동']
(82565, 13) (103139, 13) (50125, 13) (84920, 13)
(1674, 1) (1777, 1) (1605, 1) (1883, 1)
(1674, 3) (1777, 3) (1605, 3) (1883, 3)


In [ ]:
# pivot table 로 펼침 (평일)
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")

# 결측치 0
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.fillna(0)
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.fillna(0)
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.fillna(0)
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.fillna(0)

# 전치 뒤집기
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.T
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.T
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.T
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.T

# 23 개 업종 테이블
print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_seoul_weekday_seg1_corr = pd.merge(holiday, card_seoul_weekday_seg1, on='STD_DD')
card_seoul_weekday_seg2_corr = pd.merge(holiday, card_seoul_weekday_seg2,on='STD_DD')
card_seoul_weekday_seg3_corr = pd.merge(holiday, card_seoul_weekday_seg3,on='STD_DD')
card_seoul_weekday_seg4_corr = pd.merge(holiday, card_seoul_weekday_seg4,on='STD_DD')

# covid와 merge
card_seoul_weekday_seg1_corr = pd.merge(covid, card_seoul_weekday_seg1_corr, on='STD_DD')
card_seoul_weekday_seg2_corr = pd.merge(covid, card_seoul_weekday_seg2_corr,on='STD_DD')
card_seoul_weekday_seg3_corr = pd.merge(covid, card_seoul_weekday_seg3_corr,on='STD_DD')
card_seoul_weekday_seg4_corr = pd.merge(covid, card_seoul_weekday_seg4_corr,on='STD_DD')

print(card_seoul_weekday_seg1_corr.shape , card_seoul_weekday_seg2_corr.shape, card_seoul_weekday_seg3_corr.shape, card_seoul_weekday_seg4_corr.shape)

# copy
card_seoul_weekday_seg1_corr_2020 = card_seoul_weekday_seg1_corr.copy()
card_seoul_weekday_seg2_corr_2020 = card_seoul_weekday_seg2_corr.copy()
card_seoul_weekday_seg3_corr_2020 = card_seoul_weekday_seg3_corr.copy()
card_seoul_weekday_seg4_corr_2020 = card_seoul_weekday_seg4_corr.copy()

print(card_seoul_weekday_seg1_corr_2020.shape , card_seoul_weekday_seg2_corr_2020.shape, card_seoul_weekday_seg3_corr_2020.shape, card_seoul_weekday_seg4_corr_2020.shape)


(82, 22) (82, 23) (82, 23) (82, 23)
(82, 26) (82, 27) (82, 27) (82, 27)
(82, 26) (82, 27) (82, 27) (82, 27)


In [ ]:
### 2019 데이터 
# 각 cluster와 서울 카드 평일 교집합 
seoul_weekday_seg1_list = list(set(seoul_cluster1).intersection(set(card_weekday_list)))
seoul_weekday_seg2_list = list(set(seoul_cluster2).intersection(set(card_weekday_list)))
seoul_weekday_seg3_list = list(set(seoul_cluster3).intersection(set(card_weekday_list)))
seoul_weekday_seg4_list = list(set(seoul_cluster4).intersection(set(card_weekday_list)))
print(seoul_weekday_seg1_list, '\n', seoul_weekday_seg2_list, '\n', seoul_weekday_seg3_list, '\n', seoul_weekday_seg4_list)

# 서울 평일 각 seg 로 분리 데이터프레임 생성
card_seoul_weekday_seg1 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(seoul_weekday_seg1_list)]
card_seoul_weekday_seg2 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(seoul_weekday_seg2_list)] 
card_seoul_weekday_seg3 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(seoul_weekday_seg3_list)]
card_seoul_weekday_seg4 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(seoul_weekday_seg4_list)]

print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)

# 평일 각 seg별 일별, 업종별 USE_AMT 평균
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg1 = pd.DataFrame(card_seoul_weekday_seg1)

card_seoul_weekday_seg2 = card_seoul_weekday_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg2 = pd.DataFrame(card_seoul_weekday_seg2)

card_seoul_weekday_seg3 = card_seoul_weekday_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg3 = pd.DataFrame(card_seoul_weekday_seg3)

card_seoul_weekday_seg4 = card_seoul_weekday_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekday_seg4 = pd.DataFrame(card_seoul_weekday_seg4)

print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)

# 데이터 reset_index
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.reset_index()
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.reset_index()
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.reset_index()
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_seoul_weekday_seg1["MCT_CAT_CD"] = card_seoul_weekday_seg1["MCT_CAT_CD"].astype(str)
card_seoul_weekday_seg2["MCT_CAT_CD"] = card_seoul_weekday_seg2["MCT_CAT_CD"].astype(str)
card_seoul_weekday_seg3["MCT_CAT_CD"] = card_seoul_weekday_seg3["MCT_CAT_CD"].astype(str)
card_seoul_weekday_seg4["MCT_CAT_CD"] = card_seoul_weekday_seg4["MCT_CAT_CD"].astype(str)
print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)

['명동', '소공동', '회현동'] 
 ['하계1동', '중계1동', '중계4동', '중림동', '월계1동', '중계2.3동', '상계1동'] 
 ['청구동', '상계5동', '하계2동', '상계10동', '월계2동'] 
 ['공릉2동', '공릉1동', '신당5동', '을지로동', '장충동']
(100447, 13) (110571, 13) (51586, 13) (92661, 13)
(1685, 1) (1746, 1) (1553, 1) (1852, 1)
(1685, 3) (1746, 3) (1553, 3) (1852, 3)


In [ ]:
### 2019 데이터 

# pivot table 로 펼침 (평일)
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")

# 결측치 0
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.fillna(0)
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.fillna(0)
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.fillna(0)
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.fillna(0)

# 전치 뒤집기
card_seoul_weekday_seg1 = card_seoul_weekday_seg1.T
card_seoul_weekday_seg2 = card_seoul_weekday_seg2.T
card_seoul_weekday_seg3 = card_seoul_weekday_seg3.T
card_seoul_weekday_seg4 = card_seoul_weekday_seg4.T

# 23 개 업종 테이블
print(card_seoul_weekday_seg1.shape, card_seoul_weekday_seg2.shape, card_seoul_weekday_seg3.shape, card_seoul_weekday_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_seoul_weekday_seg1_corr = pd.merge(holiday, card_seoul_weekday_seg1, on='STD_DD')
card_seoul_weekday_seg2_corr = pd.merge(holiday, card_seoul_weekday_seg2,on='STD_DD')
card_seoul_weekday_seg3_corr = pd.merge(holiday, card_seoul_weekday_seg3,on='STD_DD')
card_seoul_weekday_seg4_corr = pd.merge(holiday, card_seoul_weekday_seg4,on='STD_DD')

# covid와 merge
card_seoul_weekday_seg1_corr = pd.merge(covid, card_seoul_weekday_seg1_corr, on='STD_DD')
card_seoul_weekday_seg2_corr = pd.merge(covid, card_seoul_weekday_seg2_corr,on='STD_DD')
card_seoul_weekday_seg3_corr = pd.merge(covid, card_seoul_weekday_seg3_corr,on='STD_DD')
card_seoul_weekday_seg4_corr = pd.merge(covid, card_seoul_weekday_seg4_corr,on='STD_DD')

print(card_seoul_weekday_seg1_corr.shape , card_seoul_weekday_seg2_corr.shape, card_seoul_weekday_seg3_corr.shape, card_seoul_weekday_seg4_corr.shape)

# copy
card_seoul_weekday_seg1_corr_2019 = card_seoul_weekday_seg1_corr.copy()
card_seoul_weekday_seg2_corr_2019 = card_seoul_weekday_seg2_corr.copy()
card_seoul_weekday_seg3_corr_2019 = card_seoul_weekday_seg3_corr.copy()
card_seoul_weekday_seg4_corr_2019 = card_seoul_weekday_seg4_corr.copy()

print(card_seoul_weekday_seg1_corr_2019.shape , card_seoul_weekday_seg2_corr_2019.shape, card_seoul_weekday_seg3_corr_2019.shape, card_seoul_weekday_seg4_corr_2019.shape)


(81, 22) (81, 23) (81, 23) (81, 23)
(80, 26) (80, 27) (80, 27) (80, 27)
(80, 26) (80, 27) (80, 27) (80, 27)


In [ ]:
# 최종 19, 20 합쳐진 카드 데이터
card_seoul_weekday_seg1_corr_19_20 = pd.merge(card_seoul_weekday_seg1_corr_2019, card_seoul_weekday_seg1_corr_2020, on='STD_DD')
card_seoul_weekday_seg2_corr_19_20 = pd.merge(card_seoul_weekday_seg2_corr_2019, card_seoul_weekday_seg2_corr_2020, on='STD_DD')
card_seoul_weekday_seg3_corr_19_20 = pd.merge(card_seoul_weekday_seg3_corr_2019, card_seoul_weekday_seg3_corr_2020, on='STD_DD')
card_seoul_weekday_seg4_corr_19_20 = pd.merge(card_seoul_weekday_seg4_corr_2019, card_seoul_weekday_seg4_corr_2020, on='STD_DD')

print(card_seoul_weekday_seg1_corr_19_20.shape , card_seoul_weekday_seg2_corr_19_20.shape, card_seoul_weekday_seg3_corr_19_20.shape, card_seoul_weekday_seg4_corr_19_20.shape)
card_seoul_weekday_seg1_corr_19_20

(77, 51) (77, 53) (77, 53) (77, 53)


,STD_DD,confirmed_x,day_x,weekend_x,10_x,20_x,21_x,22_x,30_x,32_x,33_x,34_x,35_x,40_x,42_x,43_x,44_x,50_x,52_x,60_x,62_x,70_x,71_x,80_x,81_x,92_x,confirmed_y,day_y,weekend_y,10_y,20_y,21_y,22_y,30_y,32_y,33_y,34_y,35_y,40_y,42_y,43_y,44_y,50_y,52_y,60_y,62_y,70_y,71_y,80_y,81_y,92_y
0,20200206,5,3,0,10408.833333,10123.067797,1227.690476,384.982143,94.00,569.875000,843.458333,295.222222,2224.805556,67208.607843,3741.891892,674.533333,1449.275362,602.014706,2069.234043,0.0,2510.240506,9971.511905,5220.696629,26822.800000,870.521277,187.764706,5,3,0,5804.691489,7093.897959,1701.659091,219.135593,0.0,412.692308,762.361111,1335.545455,1354.156250,29479.689320,1849.144928,285.200000,1154.611940,741.414286,4398.408163,0.0,1721.858974,11407.545455,4121.590909,24773.854369,802.548387,173.666667
1,20200207,3,4,0,10892.577320,11618.884615,1503.439024,378.619048,0.00,1110.111111,809.833333,1436.115385,2162.888889,80452.784314,4198.275000,324.777778,2408.589041,1143.114286,2334.630435,0.0,1936.822785,9639.317647,6010.648936,33316.424242,701.566667,738.466667,3,4,0,8017.703297,7968.560000,1976.853659,188.172414,0.0,336.333333,567.354839,924.200000,1549.538462,43154.911765,2050.608108,291.833333,1693.597222,473.958904,4161.944444,2210.0,1833.960526,10304.568182,6012.827586,28478.158416,854.608696,1059.952381
2,20200210,3,0,0,11303.969072,9624.771930,1781.244444,402.700000,75.00,1591.600000,800.321429,1851.333333,1842.074074,47097.500000,4082.197183,552.714286,1663.363636,595.041667,2401.833333,0.0,2568.467532,11701.622222,5334.193548,27642.465347,679.663043,324.812500,3,0,0,7037.505495,9388.588235,2144.425000,161.540000,0.0,667.000000,770.733333,798.375000,1696.781250,24806.705882,2049.328358,85.000000,1766.215385,425.957746,3900.480769,27.0,2458.068493,9842.722222,5715.174419,24502.650485,791.760870,189.352941
3,20200211,0,1,0,8976.170000,9285.851852,1592.575000,291.949153,93.00,831.571429,1002.200000,1326.500000,1551.933333,49994.127451,3742.383562,805.470588,2083.194444,687.455882,3245.130435,0.0,1963.051282,9173.152941,5178.182796,29002.921569,751.450549,237.461538,0,1,0,5687.000000,8033.180000,1247.300000,285.578947,0.0,597.777778,681.913043,1178.000000,2561.230769,34262.000000,2473.850746,494.142857,1696.045455,552.352941,3794.367347,0.0,1672.069444,9582.636364,5682.404494,26739.693069,812.890110,140.250000
4,20200212,1,2,0,8347.134021,10451.220000,1762.883721,349.015385,388.75,995.000000,851.296296,429.760000,934.090909,46813.058824,4071.126761,348.900000,1892.536232,616.197183,3344.104167,0.0,1784.240506,8873.813953,5175.075269,30241.823529,1550.953488,574.619048,1,2,0,5318.617978,7458.354167,996.166667,235.296296,0.0,846.200000,723.842105,3358.705882,1157.555556,33424.623762,3155.283333,112.166667,1125.446154,491.937500,3266.538462,0.0,1620.794521,7975.933333,3746.955056,25693.598039,760.689655,195.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,20200525,25,0,0,11789.635417,9240.714286,1622.386364,345.440000,994.50,531.214286,739.565217,2409.150000,2352.285714,27167.960784,3383.462500,303.153846,2295.753247,566.605634,9664.040000,0.0,3080.826667,29275.956522,5595.640449,25657.990000,760.406250,195.125000,25,0,0,1442.468750,13816.214286,1727.294118,328.000000,0.0,1791.166667,681.809524,1193.727273,1716.066667,28388.730337,3175.568966,764.764706,1453.568627,440.610169,4915.880000,0.0,448.557143,29158.468354,3924.876712,28297.430108,781.220779,484.692308
73,20200526,16,1,0,8771.371134,10690.578947,1663.088889,355.967742,0.00,647.000000,1046.692308,375.181818,1806.483871,50630.126214,4494.775000,479.750000,1707.240506,580.346667,7929.679245,0.0,1921.526316,48570.761364,6150.589474,28728.460784,906.572917,295.814815,16,1,0,2084.714286,10467.133333,1321.114286,225.312500,0.0,735.050000,1160.720000,203.384615,1902.590909,34266.936170,3245.721311,1039.750000,1324.375000,764

In [ ]:
# 서울 평일 seg1 cor_val이상 업종
print('서울 카드 평일')

card_seoul_weekday_seg1_corr40_list = [] # 상관계수 cor_val 이상인 업종
card_seoul_weekday_seg2_corr40_list = [] # 상관계수 cor_val 이상인 업종
card_seoul_weekday_seg3_corr40_list = [] # 상관계수 cor_val 이상인 업종
card_seoul_weekday_seg4_corr40_list = [] # 상관계수 cor_val 이상인 업종

print('---- 1군집 (상업지역) ----')
print(seoul_weekday_seg1_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_seoul_weekday_seg1_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 평일 군집1',type_list[i], card_seoul_weekday_seg1_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_seoul_weekday_seg1_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_seoul_weekday_seg1_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_seoul_weekday_seg1_corr40_list.append(type_list[i])
  except:
    pass

# 서울 평일 seg2 cor_val이상 업종
print('---- 2군집 (주거상권) ----')
print(seoul_weekday_seg2_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_seoul_weekday_seg2_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 평일 군집2',type_list[i], card_seoul_weekday_seg2_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_seoul_weekday_seg2_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_seoul_weekday_seg2_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_seoul_weekday_seg2_corr40_list.append(type_list[i])
  except:
    pass
print()

# 서울 평일 seg3 cor_val이상 업종
print('---- 3군집 (주거지역) ----')
print(seoul_weekday_seg3_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_seoul_weekday_seg3_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 평일 군집3',type_list[i], card_seoul_weekday_seg3_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_seoul_weekday_seg3_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_seoul_weekday_seg3_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_seoul_weekday_seg3_corr40_list.append(type_list[i])
  except:
    pass
print()

# 서울 평일 seg4 cor_val이상 업종
print('---- 4군집 (교통중심) ----')
print(seoul_weekday_seg4_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_seoul_weekday_seg4_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 평일 군집4',type_list[i], card_seoul_weekday_seg4_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_seoul_weekday_seg4_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_seoul_weekday_seg4_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_seoul_weekday_seg4_corr40_list.append(type_list[i])
  except:
    pass       
print()


서울 카드 평일
---- 1군집 (상업지역) ----
['명동', '소공동', '회현동']
서울 평일 군집1 레저용품(20) -0.5381883053364227  / semi partial :  -0.5187482984752205  코로나와 상관있음
서울 평일 군집1 유통업(40) -0.48024309267289916  / semi partial :  -0.3448726616132495 코로나와 상관없음. 우연에 의함
서울 평일 군집1 의복(42) -0.4835566638047422  / semi partial :  -0.4904566449515649  코로나와 상관있음
서울 평일 군집1 보건위생(71) -0.3819053849396763  / semi partial :  -0.3845127729445322  코로나와 상관있음
서울 평일 군집1 요식업소(80) -0.4962545379909628  / semi partial :  -0.4765684057823739  코로나와 상관있음
---- 2군집 (주거상권) ----
['하계1동', '중계1동', '중계4동', '중림동', '월계1동', '중계2.3동', '상계1동']
서울 평일 군집2 요식업소(80) -0.383422547829268  / semi partial :  -0.38489065001305695  코로나와 상관있음

---- 3군집 (주거지역) ----
['청구동', '상계5동', '하계2동', '상계10동', '월계2동']

---- 4군집 (교통중심) ----
['공릉2동', '공릉1동', '신당5동', '을지로동', '장충동']
서울 평일 군집4 요식업소(80) -0.49543235676764386  / semi partial :  -0.4537085266790283  코로나와 상관있음



### 카드 서울 휴일 (2020, 2019)

In [ ]:
# 2020년 데이터
# 각 cluster와 서울 카드 휴일 교집합 
seoul_weekend_seg1_list = list(set(seoul_cluster1).intersection(set(card_weekend_list)))
seoul_weekend_seg2_list = list(set(seoul_cluster2).intersection(set(card_weekend_list)))
seoul_weekend_seg3_list = list(set(seoul_cluster3).intersection(set(card_weekend_list)))
seoul_weekend_seg4_list = list(set(seoul_cluster4).intersection(set(card_weekend_list)))
print(seoul_weekend_seg1_list, '\n', seoul_weekend_seg2_list, '\n', seoul_weekend_seg3_list, '\n', seoul_weekend_seg4_list)

# 서울 휴일 각 seg 로 분리 데이터프레임 생성
card_seoul_weekend_seg1 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(seoul_weekend_seg1_list)]
card_seoul_weekend_seg2 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(seoul_weekend_seg2_list)] 
card_seoul_weekend_seg3 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(seoul_weekend_seg3_list)]
card_seoul_weekend_seg4 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(seoul_weekend_seg4_list)]

print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)

# 휴일 각 seg별 일별, 업종별 USE_AMT 평균
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg1 = pd.DataFrame(card_seoul_weekend_seg1)

card_seoul_weekend_seg2 = card_seoul_weekend_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg2 = pd.DataFrame(card_seoul_weekend_seg2)

card_seoul_weekend_seg3 = card_seoul_weekend_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg3 = pd.DataFrame(card_seoul_weekend_seg3)

card_seoul_weekend_seg4 = card_seoul_weekend_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg4 = pd.DataFrame(card_seoul_weekend_seg4)

print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)

# 데이터 reset_index
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.reset_index()
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.reset_index()
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.reset_index()
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_seoul_weekend_seg1["MCT_CAT_CD"] = card_seoul_weekend_seg1["MCT_CAT_CD"].astype(str)
card_seoul_weekend_seg2["MCT_CAT_CD"] = card_seoul_weekend_seg2["MCT_CAT_CD"].astype(str)
card_seoul_weekend_seg3["MCT_CAT_CD"] = card_seoul_weekend_seg3["MCT_CAT_CD"].astype(str)
card_seoul_weekend_seg4["MCT_CAT_CD"] = card_seoul_weekend_seg4["MCT_CAT_CD"].astype(str)

print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)

['필동', '명동', '소공동', '상계2동'] 
 ['하계1동', '중계본동', '중계4동', '중림동', '중계2.3동', '상계1동'] 
 ['상계10동', '월계2동', '상계5동'] 
 ['공릉2동', '신당동', '상계6.7동', '약수동', '신당5동', '을지로동', '장충동']
(39194, 13) (41707, 13) (14530, 13) (48580, 13)
(818, 1) (820, 1) (680, 1) (847, 1)
(818, 3) (820, 3) (680, 3) (847, 3)


In [ ]:
# pivot table 로 펼침 (휴일)
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")

# 결측치 0
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.fillna(0)
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.fillna(0)
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.fillna(0)
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.fillna(0)

# 전치 뒤집기
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.T
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.T
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.T
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.T

# card 업종 23개 pivot table
print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_seoul_weekend_seg1_corr = pd.merge(holiday, card_seoul_weekend_seg1, on='STD_DD')
card_seoul_weekend_seg2_corr = pd.merge(holiday, card_seoul_weekend_seg2,on='STD_DD')
card_seoul_weekend_seg3_corr = pd.merge(holiday, card_seoul_weekend_seg3,on='STD_DD')
card_seoul_weekend_seg4_corr = pd.merge(holiday, card_seoul_weekend_seg4,on='STD_DD')

# covid와 merge
card_seoul_weekend_seg1_corr = pd.merge(covid, card_seoul_weekend_seg1_corr, on='STD_DD')
card_seoul_weekend_seg2_corr = pd.merge(covid, card_seoul_weekend_seg2_corr,on='STD_DD')
card_seoul_weekend_seg3_corr = pd.merge(covid, card_seoul_weekend_seg3_corr,on='STD_DD')
card_seoul_weekend_seg4_corr = pd.merge(covid, card_seoul_weekend_seg4_corr,on='STD_DD')

print(card_seoul_weekend_seg1_corr.shape , card_seoul_weekend_seg2_corr.shape, card_seoul_weekend_seg3_corr.shape, card_seoul_weekend_seg4_corr.shape)

# copy
card_seoul_weekend_seg1_corr_2020 = card_seoul_weekend_seg1_corr.copy()
card_seoul_weekend_seg2_corr_2020 = card_seoul_weekend_seg2_corr.copy()
card_seoul_weekend_seg3_corr_2020 = card_seoul_weekend_seg3_corr.copy()
card_seoul_weekend_seg4_corr_2020 = card_seoul_weekend_seg4_corr.copy()

print(card_seoul_weekend_seg1_corr_2020.shape , card_seoul_weekend_seg2_corr_2020.shape, card_seoul_weekend_seg3_corr_2020.shape, card_seoul_weekend_seg4_corr_2020.shape)

(39, 23) (39, 23) (39, 22) (39, 23)
(39, 27) (39, 27) (39, 26) (39, 27)
(39, 27) (39, 27) (39, 26) (39, 27)


In [ ]:
# 2019년 데이터
# 각 cluster와 서울 카드 휴일 교집합 
seoul_weekend_seg1_list = list(set(seoul_cluster1).intersection(set(card_weekend_list)))
seoul_weekend_seg2_list = list(set(seoul_cluster2).intersection(set(card_weekend_list)))
seoul_weekend_seg3_list = list(set(seoul_cluster3).intersection(set(card_weekend_list)))
seoul_weekend_seg4_list = list(set(seoul_cluster4).intersection(set(card_weekend_list)))
print(seoul_weekend_seg1_list, '\n', seoul_weekend_seg2_list, '\n', seoul_weekend_seg3_list, '\n', seoul_weekend_seg4_list)

# 서울 휴일 각 seg 로 분리 데이터프레임 생성
card_seoul_weekend_seg1 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(seoul_weekend_seg1_list)]
card_seoul_weekend_seg2 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(seoul_weekend_seg2_list)] 
card_seoul_weekend_seg3 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(seoul_weekend_seg3_list)]
card_seoul_weekend_seg4 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(seoul_weekend_seg4_list)]

print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)

# 휴일 각 seg별 일별, 업종별 USE_AMT 평균
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg1 = pd.DataFrame(card_seoul_weekend_seg1)

card_seoul_weekend_seg2 = card_seoul_weekend_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg2 = pd.DataFrame(card_seoul_weekend_seg2)

card_seoul_weekend_seg3 = card_seoul_weekend_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg3 = pd.DataFrame(card_seoul_weekend_seg3)

card_seoul_weekend_seg4 = card_seoul_weekend_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_seoul_weekend_seg4 = pd.DataFrame(card_seoul_weekend_seg4)

print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)

# 데이터 reset_index
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.reset_index()
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.reset_index()
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.reset_index()
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_seoul_weekend_seg1["MCT_CAT_CD"] = card_seoul_weekend_seg1["MCT_CAT_CD"].astype(str)
card_seoul_weekend_seg2["MCT_CAT_CD"] = card_seoul_weekend_seg2["MCT_CAT_CD"].astype(str)
card_seoul_weekend_seg3["MCT_CAT_CD"] = card_seoul_weekend_seg3["MCT_CAT_CD"].astype(str)
card_seoul_weekend_seg4["MCT_CAT_CD"] = card_seoul_weekend_seg4["MCT_CAT_CD"].astype(str)

print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)

['필동', '명동', '소공동', '상계2동'] 
 ['하계1동', '중계본동', '중계4동', '중림동', '중계2.3동', '상계1동'] 
 ['상계10동', '월계2동', '상계5동'] 
 ['공릉2동', '신당동', '상계6.7동', '약수동', '신당5동', '을지로동', '장충동']
(47522, 13) (44584, 13) (14630, 13) (53519, 13)
(818, 1) (815, 1) (674, 1) (839, 1)
(818, 3) (815, 3) (674, 3) (839, 3)


In [ ]:
# pivot table 로 펼침 (휴일)
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")

# 결측치 0
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.fillna(0)
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.fillna(0)
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.fillna(0)
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.fillna(0)

# 전치 뒤집기
card_seoul_weekend_seg1 = card_seoul_weekend_seg1.T
card_seoul_weekend_seg2 = card_seoul_weekend_seg2.T
card_seoul_weekend_seg3 = card_seoul_weekend_seg3.T
card_seoul_weekend_seg4 = card_seoul_weekend_seg4.T

# card 업종 23개 pivot table
print(card_seoul_weekend_seg1.shape, card_seoul_weekend_seg2.shape, card_seoul_weekend_seg3.shape, card_seoul_weekend_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_seoul_weekend_seg1_corr = pd.merge(holiday, card_seoul_weekend_seg1, on='STD_DD')
card_seoul_weekend_seg2_corr = pd.merge(holiday, card_seoul_weekend_seg2,on='STD_DD')
card_seoul_weekend_seg3_corr = pd.merge(holiday, card_seoul_weekend_seg3,on='STD_DD')
card_seoul_weekend_seg4_corr = pd.merge(holiday, card_seoul_weekend_seg4,on='STD_DD')

# covid와 merge
card_seoul_weekend_seg1_corr = pd.merge(covid, card_seoul_weekend_seg1_corr, on='STD_DD')
card_seoul_weekend_seg2_corr = pd.merge(covid, card_seoul_weekend_seg2_corr,on='STD_DD')
card_seoul_weekend_seg3_corr = pd.merge(covid, card_seoul_weekend_seg3_corr,on='STD_DD')
card_seoul_weekend_seg4_corr = pd.merge(covid, card_seoul_weekend_seg4_corr,on='STD_DD')

print(card_seoul_weekend_seg1_corr.shape , card_seoul_weekend_seg2_corr.shape, card_seoul_weekend_seg3_corr.shape, card_seoul_weekend_seg4_corr.shape)

# copy
card_seoul_weekend_seg1_corr_2019 = card_seoul_weekend_seg1_corr.copy()
card_seoul_weekend_seg2_corr_2019 = card_seoul_weekend_seg2_corr.copy()
card_seoul_weekend_seg3_corr_2019 = card_seoul_weekend_seg3_corr.copy()
card_seoul_weekend_seg4_corr_2019 = card_seoul_weekend_seg4_corr.copy()

print(card_seoul_weekend_seg1_corr_2019.shape , card_seoul_weekend_seg2_corr_2019.shape, card_seoul_weekend_seg3_corr_2019.shape, card_seoul_weekend_seg4_corr_2019.shape)

(39, 22) (39, 23) (39, 23) (39, 23)
(39, 26) (39, 27) (39, 27) (39, 27)
(39, 26) (39, 27) (39, 27) (39, 27)


In [ ]:
# 최종 19, 20 합쳐진 카드 데이터 - 휴일
card_seoul_weekend_seg1_corr_19_20 = pd.merge(card_seoul_weekend_seg1_corr_2019, card_seoul_weekend_seg1_corr_2020, on='STD_DD')
card_seoul_weekend_seg2_corr_19_20 = pd.merge(card_seoul_weekend_seg2_corr_2019, card_seoul_weekend_seg2_corr_2020, on='STD_DD')
card_seoul_weekend_seg3_corr_19_20 = pd.merge(card_seoul_weekend_seg3_corr_2019, card_seoul_weekend_seg3_corr_2020, on='STD_DD')
card_seoul_weekend_seg4_corr_19_20 = pd.merge(card_seoul_weekend_seg4_corr_2019, card_seoul_weekend_seg4_corr_2020, on='STD_DD')

print(card_seoul_weekend_seg1_corr_19_20.shape , card_seoul_weekend_seg2_corr_19_20.shape, card_seoul_weekend_seg3_corr_19_20.shape, card_seoul_weekend_seg4_corr_19_20.shape)
card_seoul_weekend_seg1_corr_19_20

(34, 52) (34, 53) (34, 52) (34, 53)


,STD_DD,confirmed_x,day_x,weekend_x,10_x,20_x,21_x,22_x,30_x,32_x,33_x,34_x,35_x,40_x,42_x,43_x,44_x,50_x,52_x,60_x,62_x,70_x,71_x,80_x,81_x,92_x,confirmed_y,day_y,weekend_y,10_y,20_y,21_y,22_y,30_y,31,32_y,33_y,34_y,35_y,40_y,42_y,43_y,44_y,50_y,52_y,60_y,62_y,70_y,71_y,80_y,81_y,92_y
0,20200201,4,5,1,8474.348315,19408.076923,1219.836364,1195.355263,73.000000,516.714286,794.686275,818.440000,4119.555556,40948.467213,4713.164179,187.600000,1514.440678,1047.955882,364.391304,77.000000,707.883117,5790.574257,4490.198020,22884.590551,900.245283,243.312500,4,5,1,8490.659091,12363.076923,1103.898305,243.563380,0.000000,0.0,120.857143,537.208333,894.636364,5453.928571,32575.945312,2703.234375,393.000000,1319.468750,759.042857,190.043478,205.500000,498.750000,5834.902913,4636.863158,20852.147287,532.971963,197.285714
1,20200202,1,6,1,7980.494382,15551.307692,1033.840000,839.426471,0.000000,131.000000,573.818182,212.100000,1989.250000,30830.952381,3559.393443,242.000000,1347.862069,423.537037,272.050000,0.000000,350.656716,915.140351,3367.966292,19372.862903,712.164835,218.600000,1,6,1,7424.569767,13387.816327,1076.886792,258.016129,137.000000,0.0,101.571429,517.357143,132.764706,6155.000000,24059.222222,2582.428571,442.000000,1816.285714,471.098361,198.684211,0.000000,164.046154,1602.729412,3574.652632,16653.255814,568.961538,121.500000
2,20200208,0,5,1,10360.258427,17317.849057,1068.242424,821.053333,2060.000000,234.000000,706.333333,1440.083333,1056.636364,47521.642857,5009.446154,188.750000,2218.634921,1084.500000,314.200000,68.000000,703.935897,6375.574468,5669.831683,25165.929134,582.104762,198.260870,0,5,1,10577.324675,12218.274510,905.903226,327.300000,0.000000,0.0,135.142857,568.562500,941.043478,718.750000,10267.870968,2911.015873,124.400000,1696.873016,690.000000,274.076923,139.666667,586.430556,5418.372549,5071.940476,20312.658915,468.000000,377.846154
3,20200209,0,6,1,12288.488372,17135.288462,1038.472727,734.313433,0.000000,85.000000,303.714286,774.923077,3379.000000,44556.566667,4848.796610,67.000000,2086.984375,718.147541,159.950000,0.000000,415.385714,1106.030303,4729.061856,21130.344000,496.514851,168.625000,0,6,1,6434.525000,13375.470588,901.403846,305.916667,0.000000,0.0,110.750000,355.641026,1158.250000,15705.000000,10343.292683,2642.285714,256.000000,2461.462963,459.083333,202.000000,0.000000,268.420290,902.457143,4959.000000,17982.690476,336.841584,432.666667
4,20200215,0,5,1,10647.629213,16351.807692,890.276923,819.106667,4972.000000,270.750000,760.125000,435.689655,641.000000,41633.048387,4932.055556,80.000000,1878.382353,669.452055,161.172414,1963.600000,876.041667,5644.767677,5666.803922,26370.054264,596.931373,360.750000,0,5,1,5210.597561,15284.019231,1086.769231,565.101266,619.000000,0.0,322.888889,617.285714,1771.208333,2492.250000,29097.388889,3643.184615,413.300000,2672.417910,654.088235,219.125000,774.000000,280.430556,5114.525253,4835.000000,23450.793651,468.644860,230.105263
5,20200216,0,6,1,11103.883721,17501.169811,757.440678,972.611940,0.000000,142.500000,492.692308,266.866667,1875.500000,35602.365854,4906.852459,147.000000,1675.900000,538.166667,180.187500,0.000000,466.736111,883.785714,4705.242105,20396.923077,644.825688,261.090909,0,6,1,5760.684211,13578.360000,1285.781818,373.421875,0.000000,0.0,313.000000,368.309524,266.000000,9058.500000,24806.581967,1929.368421,950.000000,1379.547170,374.614035,224.529412,0.000000,147.271429,1273.236111,3294.931818,17311.582677,348.117647,188.750000
6,20200222,100,5,1,10892.589474,18485.054545,1100.850000,823.405405,94.500000,447.800000,764.122807,738.657143,1080.538462,46457.901639,5164.885714,106.142857,2484.507246,590.890411,432.952381,287.000000,790.126761,6819.357143,5303.778846,28418.056452,549.342593,182.517241,100,5,1,6951.908046,9709.920000,1240.490909,292.372881,1944.333333,0.0,100.222222,642.333333,559.500000,309.600000,23655.992248,2206.870968,40.000000,1791.265625,528.000000,995.318182,165.500000,262.073529,5450.690000,3926.117021,17925.629921,

In [ ]:
# 서울 휴일 seg1 cor_val이상 업종
print('서울 카드 휴일')

card_seoul_weekend_seg1_corr40_list = [] # 상관계수 cor_val 이상인 업종
card_seoul_weekend_seg2_corr40_list = [] # 상관계수 cor_val 이상인 업종
card_seoul_weekend_seg3_corr40_list = [] # 상관계수 cor_val 이상인 업종
card_seoul_weekend_seg4_corr40_list = [] # 상관계수 cor_val 이상인 업종

print('---- 1군집 (상업지역) ----')
print(seoul_weekend_seg1_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_seoul_weekend_seg1_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 휴일 군집1',type_list[i], card_seoul_weekend_seg1_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_seoul_weekend_seg1_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_seoul_weekend_seg1_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_seoul_weekend_seg1_corr40_list.append(type_list[i])
  except:
    pass

# 서울 휴일 seg2 cor_val이상 업종
print('---- 2군집 (주거상권) ----')
print(seoul_weekend_seg2_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_seoul_weekend_seg2_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 휴일 군집2',type_list[i], card_seoul_weekend_seg2_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_seoul_weekend_seg2_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_seoul_weekend_seg2_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_seoul_weekend_seg2_corr40_list.append(type_list[i])
        
  except:
    pass
print()

# 서울 휴일 seg3 cor_val이상 업종
print('---- 3군집 (주거지역) ----')
print(seoul_weekend_seg3_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_seoul_weekend_seg3_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 휴일 군집3',type_list[i], card_seoul_weekend_seg3_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_seoul_weekend_seg3_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_seoul_weekend_seg3_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_seoul_weekend_seg3_corr40_list.append(type_list[i])
  except:
    pass
print()

# 서울 휴일 seg4 cor_val이상 업종
print('---- 4군집 (교통중심) ----')
print(seoul_weekend_seg4_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_seoul_weekend_seg4_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 휴일 군집4',type_list[i], card_seoul_weekend_seg4_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_seoul_weekend_seg4_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_seoul_weekend_seg4_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_seoul_weekend_seg4_corr40_list.append(type_list[i])
  except:
    pass       
print()


서울 카드 휴일
---- 1군집 (상업지역) ----
['필동', '명동', '소공동', '상계2동']
서울 휴일 군집1 레저용품(20) -0.5088041318878149  / semi partial :  -0.5418873937179105  코로나와 상관있음
서울 휴일 군집1 레저업소(21) -0.36928916060002703  / semi partial :  -0.39326274977241876  코로나와 상관있음
서울 휴일 군집1 유통업(40) -0.3807891969489965  / semi partial :  -0.36924267834027713  코로나와 상관있음
서울 휴일 군집1 의복(42) -0.617420075573389  / semi partial :  -0.6523091758423918  코로나와 상관있음
서울 휴일 군집1 서적문구(50) -0.5893102873131781  / semi partial :  -0.5445939893999954  코로나와 상관있음
서울 휴일 군집1 요식업소(80) -0.5193516830130954  / semi partial :  -0.4556261497455004  코로나와 상관있음
---- 2군집 (주거상권) ----
['하계1동', '중계본동', '중계4동', '중림동', '중계2.3동', '상계1동']
서울 휴일 군집2 숙박(10) 0.37902505407005005  / semi partial :  0.36605465494040595  코로나와 상관있음
서울 휴일 군집2 전기(31) 0.5902957597088287  / semi partial :  0.5854376951340164  코로나와 상관있음
서울 휴일 군집2 유통업(40) -0.35562396911062083  / semi partial :  -0.40757489896082094  코로나와 상관있음
서울 휴일 군집2 보건위생(71) -0.377334523218468  / semi partial :  -0.4614176259235484

### 카드 대구 평일 (2020, 2019)

In [ ]:
# 2020년도
# 각 cluster와 대구 카드 평일 교집합 
daegu_weekday_seg1_list = list(set(daegu_cluster1).intersection(set(card_weekday_list)))
daegu_weekday_seg2_list = list(set(daegu_cluster2).intersection(set(card_weekday_list)))
daegu_weekday_seg3_list = list(set(daegu_cluster3).intersection(set(card_weekday_list)))
daegu_weekday_seg4_list = list(set(daegu_cluster4).intersection(set(card_weekday_list)))
print(daegu_weekday_seg1_list, '\n', daegu_weekday_seg2_list, '\n', daegu_weekday_seg3_list, '\n', daegu_weekday_seg4_list)

# 대구 평일 각 군집. 분리 데이터프레임 생성
card_daegu_weekday_seg1 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(daegu_weekday_seg1_list)]
card_daegu_weekday_seg2 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(daegu_weekday_seg2_list)] 
card_daegu_weekday_seg3 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(daegu_weekday_seg3_list)]
card_daegu_weekday_seg4 = card2020_weekday[card2020_weekday['HDONG_NM'].isin(daegu_weekday_seg4_list)]

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)

# 대구 평일 각 군집별 일별, 업종별 USE_AMT 평균
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg1 = pd.DataFrame(card_daegu_weekday_seg1)

card_daegu_weekday_seg2 = card_daegu_weekday_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg2 = pd.DataFrame(card_daegu_weekday_seg2)

card_daegu_weekday_seg3 = card_daegu_weekday_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg3 = pd.DataFrame(card_daegu_weekday_seg3)

card_daegu_weekday_seg4 = card_daegu_weekday_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg4 = pd.DataFrame(card_daegu_weekday_seg4)

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)

# 데이터 reset_index
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.reset_index()
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.reset_index()
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.reset_index()
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_daegu_weekday_seg1["MCT_CAT_CD"] = card_daegu_weekday_seg1["MCT_CAT_CD"].astype(str)
card_daegu_weekday_seg2["MCT_CAT_CD"] = card_daegu_weekday_seg2["MCT_CAT_CD"].astype(str)
card_daegu_weekday_seg3["MCT_CAT_CD"] = card_daegu_weekday_seg3["MCT_CAT_CD"].astype(str)
card_daegu_weekday_seg4["MCT_CAT_CD"] = card_daegu_weekday_seg4["MCT_CAT_CD"].astype(str)

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)

['두산동', '범어1동', '성내1동', '대봉1동', '대신동', '범어2동', '성내2동', '삼덕동'] 
 ['남산4동', '만촌3동', '범어3동', '범물1동', '범어4동'] 
 ['남산3동', '상동', '수성1가동', '수성4가동', '대봉2동'] 
 ['만촌2동', '만촌1동', '남산2동', '수성2.3가동', '황금2동', '남산1동']
(116703, 13) (48941, 13) (37527, 13) (59204, 13)
(1816, 1) (1407, 1) (1422, 1) (1653, 1)
(1816, 3) (1407, 3) (1422, 3) (1653, 3)


In [ ]:
# pivot table 로 펼침 (평일)
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")

# 결측치 0
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.fillna(0)
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.fillna(0)
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.fillna(0)
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.fillna(0)

# 전치 뒤집기
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.T
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.T
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.T
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.T

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_daegu_weekday_seg1_corr = pd.merge(holiday, card_daegu_weekday_seg1, on='STD_DD')
card_daegu_weekday_seg2_corr = pd.merge(holiday, card_daegu_weekday_seg2,on='STD_DD')
card_daegu_weekday_seg3_corr = pd.merge(holiday, card_daegu_weekday_seg3,on='STD_DD')
card_daegu_weekday_seg4_corr = pd.merge(holiday, card_daegu_weekday_seg4,on='STD_DD')

# covid와 merge
card_daegu_weekday_seg1_corr = pd.merge(covid, card_daegu_weekday_seg1_corr, on='STD_DD')
card_daegu_weekday_seg2_corr = pd.merge(covid, card_daegu_weekday_seg2_corr,on='STD_DD')
card_daegu_weekday_seg3_corr = pd.merge(covid, card_daegu_weekday_seg3_corr,on='STD_DD')
card_daegu_weekday_seg4_corr = pd.merge(covid, card_daegu_weekday_seg4_corr,on='STD_DD')

# card 업종 23개 pivot table + covid, holiday
print(card_daegu_weekday_seg1_corr.shape , card_daegu_weekday_seg2_corr.shape, card_daegu_weekday_seg3_corr.shape, card_daegu_weekday_seg4_corr.shape)

# copy
card_daegu_weekday_seg1_corr_2020 = card_daegu_weekday_seg1_corr.copy()
card_daegu_weekday_seg2_corr_2020 = card_daegu_weekday_seg2_corr.copy()
card_daegu_weekday_seg3_corr_2020 = card_daegu_weekday_seg3_corr.copy()
card_daegu_weekday_seg4_corr_2020 = card_daegu_weekday_seg4_corr.copy()

print(card_daegu_weekday_seg1_corr_2020.shape , card_daegu_weekday_seg2_corr_2020.shape, card_daegu_weekday_seg3_corr_2020.shape, card_daegu_weekday_seg4_corr_2020.shape)

(82, 23) (82, 22) (82, 23) (82, 23)
(82, 27) (82, 26) (82, 27) (82, 27)
(82, 27) (82, 26) (82, 27) (82, 27)


In [ ]:
# 2019년도
# 각 cluster와 대구 카드 평일 교집합 
daegu_weekday_seg1_list = list(set(daegu_cluster1).intersection(set(card_weekday_list)))
daegu_weekday_seg2_list = list(set(daegu_cluster2).intersection(set(card_weekday_list)))
daegu_weekday_seg3_list = list(set(daegu_cluster3).intersection(set(card_weekday_list)))
daegu_weekday_seg4_list = list(set(daegu_cluster4).intersection(set(card_weekday_list)))
print(daegu_weekday_seg1_list, '\n', daegu_weekday_seg2_list, '\n', daegu_weekday_seg3_list, '\n', daegu_weekday_seg4_list)

# 대구 평일 각 군집. 분리 데이터프레임 생성
card_daegu_weekday_seg1 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(daegu_weekday_seg1_list)]
card_daegu_weekday_seg2 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(daegu_weekday_seg2_list)] 
card_daegu_weekday_seg3 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(daegu_weekday_seg3_list)]
card_daegu_weekday_seg4 = card2019_weekday[card2019_weekday['HDONG_NM'].isin(daegu_weekday_seg4_list)]

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)

# 대구 평일 각 군집별 일별, 업종별 USE_AMT 평균
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg1 = pd.DataFrame(card_daegu_weekday_seg1)

card_daegu_weekday_seg2 = card_daegu_weekday_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg2 = pd.DataFrame(card_daegu_weekday_seg2)

card_daegu_weekday_seg3 = card_daegu_weekday_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg3 = pd.DataFrame(card_daegu_weekday_seg3)

card_daegu_weekday_seg4 = card_daegu_weekday_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekday_seg4 = pd.DataFrame(card_daegu_weekday_seg4)

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)

# 데이터 reset_index
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.reset_index()
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.reset_index()
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.reset_index()
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_daegu_weekday_seg1["MCT_CAT_CD"] = card_daegu_weekday_seg1["MCT_CAT_CD"].astype(str)
card_daegu_weekday_seg2["MCT_CAT_CD"] = card_daegu_weekday_seg2["MCT_CAT_CD"].astype(str)
card_daegu_weekday_seg3["MCT_CAT_CD"] = card_daegu_weekday_seg3["MCT_CAT_CD"].astype(str)
card_daegu_weekday_seg4["MCT_CAT_CD"] = card_daegu_weekday_seg4["MCT_CAT_CD"].astype(str)

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)

['두산동', '범어1동', '성내1동', '대봉1동', '대신동', '범어2동', '성내2동', '삼덕동'] 
 ['남산4동', '만촌3동', '범어3동', '범물1동', '범어4동'] 
 ['남산3동', '상동', '수성1가동', '수성4가동', '대봉2동'] 
 ['만촌2동', '만촌1동', '남산2동', '수성2.3가동', '황금2동', '남산1동']
(138836, 13) (54256, 13) (40593, 13) (66193, 13)
(1822, 1) (1395, 1) (1431, 1) (1690, 1)
(1822, 3) (1395, 3) (1431, 3) (1690, 3)


In [ ]:
# pivot table 로 펼침 (평일)
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")

# 결측치 0
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.fillna(0)
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.fillna(0)
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.fillna(0)
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.fillna(0)

# 전치 뒤집기
card_daegu_weekday_seg1 = card_daegu_weekday_seg1.T
card_daegu_weekday_seg2 = card_daegu_weekday_seg2.T
card_daegu_weekday_seg3 = card_daegu_weekday_seg3.T
card_daegu_weekday_seg4 = card_daegu_weekday_seg4.T

print(card_daegu_weekday_seg1.shape, card_daegu_weekday_seg2.shape, card_daegu_weekday_seg3.shape, card_daegu_weekday_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_daegu_weekday_seg1_corr = pd.merge(holiday, card_daegu_weekday_seg1, on='STD_DD')
card_daegu_weekday_seg2_corr = pd.merge(holiday, card_daegu_weekday_seg2,on='STD_DD')
card_daegu_weekday_seg3_corr = pd.merge(holiday, card_daegu_weekday_seg3,on='STD_DD')
card_daegu_weekday_seg4_corr = pd.merge(holiday, card_daegu_weekday_seg4,on='STD_DD')

# covid와 merge
card_daegu_weekday_seg1_corr = pd.merge(covid, card_daegu_weekday_seg1_corr, on='STD_DD')
card_daegu_weekday_seg2_corr = pd.merge(covid, card_daegu_weekday_seg2_corr,on='STD_DD')
card_daegu_weekday_seg3_corr = pd.merge(covid, card_daegu_weekday_seg3_corr,on='STD_DD')
card_daegu_weekday_seg4_corr = pd.merge(covid, card_daegu_weekday_seg4_corr,on='STD_DD')

# card 업종 23개 pivot table + covid, holiday
print(card_daegu_weekday_seg1_corr.shape , card_daegu_weekday_seg2_corr.shape, card_daegu_weekday_seg3_corr.shape, card_daegu_weekday_seg4_corr.shape)

# copy
card_daegu_weekday_seg1_corr_2019 = card_daegu_weekday_seg1_corr.copy()
card_daegu_weekday_seg2_corr_2019 = card_daegu_weekday_seg2_corr.copy()
card_daegu_weekday_seg3_corr_2019 = card_daegu_weekday_seg3_corr.copy()
card_daegu_weekday_seg4_corr_2019 = card_daegu_weekday_seg4_corr.copy()

print(card_daegu_weekday_seg1_corr_2019.shape , card_daegu_weekday_seg2_corr_2019.shape, card_daegu_weekday_seg3_corr_2019.shape, card_daegu_weekday_seg4_corr_2019.shape)

(81, 23) (81, 22) (81, 23) (81, 23)
(80, 27) (80, 26) (80, 27) (80, 27)
(80, 27) (80, 26) (80, 27) (80, 27)


In [ ]:
# 최종 19, 20 합쳐진 카드 데이터 - 대구 평일
card_daegu_weekday_seg1_corr_19_20 = pd.merge(card_daegu_weekday_seg1_corr_2019, card_daegu_weekday_seg1_corr_2020, on='STD_DD')
card_daegu_weekday_seg2_corr_19_20 = pd.merge(card_daegu_weekday_seg2_corr_2019, card_daegu_weekday_seg2_corr_2020, on='STD_DD')
card_daegu_weekday_seg3_corr_19_20 = pd.merge(card_daegu_weekday_seg3_corr_2019, card_daegu_weekday_seg3_corr_2020, on='STD_DD')
card_daegu_weekday_seg4_corr_19_20 = pd.merge(card_daegu_weekday_seg4_corr_2019, card_daegu_weekday_seg4_corr_2020, on='STD_DD')

print(card_daegu_weekday_seg1_corr_19_20.shape , card_daegu_weekday_seg2_corr_19_20.shape, card_daegu_weekday_seg3_corr_19_20.shape, card_daegu_weekday_seg4_corr_19_20.shape)
card_daegu_weekday_seg1_corr_19_20

(77, 53) (77, 51) (77, 53) (77, 53)


,STD_DD,confirmed_x,day_x,weekend_x,10_x,20_x,21_x,22_x,30_x,31_x,32_x,33_x,34_x,35_x,40_x,42_x,43_x,44_x,50_x,52_x,60_x,62_x,70_x,71_x,80_x,81_x,92_x,confirmed_y,day_y,weekend_y,10_y,20_y,21_y,22_y,30_y,31_y,32_y,33_y,34_y,35_y,40_y,42_y,43_y,44_y,50_y,52_y,60_y,62_y,70_y,71_y,80_y,81_y,92_y
0,20200206,5,3,0,815.701754,1792.037736,668.848837,669.365591,2322.500000,81.0,485.500000,1443.530120,826.882353,3725.727273,6956.500000,2813.408602,646.000000,1926.709091,656.029412,7121.157895,1867.833333,309.629213,10231.339394,1214.686567,5416.309278,404.006667,125.888889,5,3,0,693.329114,664.772727,483.370787,613.904762,0.000000,511.0,242.500000,1334.142857,354.576923,1300.400000,6242.899471,1899.988889,626.083333,1824.867925,357.383333,7272.000000,1432.000000,159.250000,7446.529762,1021.741071,4342.964286,431.937888,149.454545
1,20200207,3,4,0,914.060606,1731.411765,757.533333,832.564356,3300.250000,900.0,2782.400000,1462.076923,746.225806,2162.916667,6217.546392,3228.530000,1287.941176,1334.509091,567.485294,8209.075472,2270.750000,266.103774,10559.588957,1161.669173,6573.331633,682.794702,214.789474,3,4,0,789.909091,877.909091,624.038462,506.380282,5518.000000,0.0,583.166667,1387.416667,587.958333,1501.250000,6277.943590,2288.712871,1253.400000,691.485294,450.492754,8581.780000,2056.428571,177.415730,10802.830409,933.543860,5664.281915,523.951220,143.291667
2,20200210,3,0,0,997.486486,1250.552632,932.022727,745.063830,3967.333333,81.0,776.625000,1452.439024,1094.560000,3221.857143,5358.196891,2312.931373,2143.277778,1244.754386,472.660714,7810.380000,6010.400000,370.750000,8789.327381,949.763359,4581.256545,579.006329,165.937500,3,0,0,901.986111,1349.693878,566.730000,653.573529,5004.666667,0.0,697.454545,1596.444444,1357.080000,3151.250000,2612.902174,1888.484536,1992.000000,1195.796296,421.718750,6739.301887,1046.200000,278.047619,8868.245614,1051.864000,4208.915842,482.257862,138.555556
3,20200211,0,1,0,1137.294118,1317.586957,789.000000,774.776596,5410.000000,0.0,410.200000,1562.569620,254.500000,1060.909091,5908.898990,2450.460784,1823.187500,1042.344828,505.895522,8244.717391,898.666667,145.305263,10431.335404,1113.207207,4666.894472,503.811321,130.375000,0,1,0,829.120000,1104.160000,654.258427,347.720000,3478.000000,754.0,1245.000000,1321.457831,166.947368,1932.444444,6613.768041,2070.030000,3002.916667,1110.655738,396.295082,6845.535714,1618.571429,210.721519,9817.459627,915.983051,4640.170984,486.221519,200.423077
4,20200212,1,2,0,1010.623377,1595.418605,1035.458824,795.308511,3354.000000,525.0,283.900000,1500.261905,458.960000,2318.846154,7269.143590,2367.295238,836.153846,903.000000,622.630769,8114.220000,1309.750000,202.258427,9000.569620,1325.560000,5296.854167,547.358974,168.642857,1,2,0,611.013333,924.477273,761.000000,428.058824,4295.000000,0.0,363.666667,1315.607595,199.777778,871.111111,6851.178010,1858.152174,1009.384615,987.060000,578.400000,7679.882353,2351.400000,272.793103,8758.806250,677.758065,4482.331606,333.198718,106.227273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,20200525,25,0,0,2087.575000,1210.179487,653.859155,1110.822222,3076.000000,0.0,635.666667,1571.414634,619.160000,1968.500000,3772.171123,1860.725490,1042.950000,1127.250000,370.015625,7842.666667,629.500000,335.698925,9582.579268,977.937008,3999.722513,416.936709,268.120000,25,0,0,644.434783,1288.660377,913.882353,352.152542,6864.500000,4721.0,1361.636364,1323.302632,144.368421,4909.200000,4999.276836,2363.290000,837.263158,955.530303,748.533333,6774.285714,2632.000000,328.837209,9278.071856,1123.352941,3725.873626,629.696203,284.838710
73,20200526,16,1,0,1378.883117,906.022222,685.148148,573.891304,585.333333,5720.0,832.500000,1570.282353,194.700000,567.800000,6000.129353,2869.034188,748.105263,1494.753425,485.000000,8112.196078,499.500000,319.325843,9065.801205,998.3

In [ ]:
# 대구 평일 seg1 cor_val이상 업종
print('대구 카드 평일')

card_daegu_weekday_seg1_corr40_list = [] # 상관계수 cor_val 이상인 업종
card_daegu_weekday_seg2_corr40_list = [] # 상관계수 cor_val 이상인 업종
card_daegu_weekday_seg3_corr40_list = [] # 상관계수 cor_val 이상인 업종
card_daegu_weekday_seg4_corr40_list = [] # 상관계수 cor_val 이상인 업종

print('---- 1군집 (상업지역) ----')
print(daegu_weekday_seg1_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_daegu_weekday_seg1_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 평일 군집1',type_list[i], card_daegu_weekday_seg1_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_daegu_weekday_seg1_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_daegu_weekday_seg1_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_daegu_weekday_seg1_corr40_list.append(type_list[i])
  except:
    pass

# 대구 평일 seg2 cor_val이상 업종
print('---- 2군집 (주거상권) ----')
print(daegu_weekday_seg2_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_daegu_weekday_seg2_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 평일 군집2',type_list[i], card_daegu_weekday_seg2_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_daegu_weekday_seg2_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_daegu_weekday_seg2_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_daegu_weekday_seg2_corr40_list.append(type_list[i])
        
  except:
    pass
print()

# 대구 평일 seg3 cor_val이상 업종
print('---- 3군집 (주거지역) ----')
print(daegu_weekday_seg3_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_daegu_weekday_seg3_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 평일 군집3',type_list[i], card_daegu_weekday_seg3_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_daegu_weekday_seg3_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_daegu_weekday_seg3_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_daegu_weekday_seg3_corr40_list.append(type_list[i])
  except:
    pass
print()

# 대구 평일 seg4 cor_val이상 업종
print('---- 4군집 (교통중심) ----')
print(daegu_weekday_seg4_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_daegu_weekday_seg4_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 평일 군집4',type_list[i], card_daegu_weekday_seg4_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_daegu_weekday_seg4_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_daegu_weekday_seg4_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_daegu_weekday_seg4_corr40_list.append(type_list[i])
  except:
    pass       
print()


대구 카드 평일
---- 1군집 (상업지역) ----
['두산동', '범어1동', '성내1동', '대봉1동', '대신동', '범어2동', '성내2동', '삼덕동']
대구 평일 군집1 레저업소(21) -0.5272138520306148  / semi partial :  -0.5302141687001395  코로나와 상관있음
대구 평일 군집1 연료판매(33) -0.5366153447981846  / semi partial :  -0.4463610766363626  코로나와 상관있음
대구 평일 군집1 유통업(40) -0.5500925896873364  / semi partial :  -0.503634240418633  코로나와 상관있음
대구 평일 군집1 사무통신(52) -0.6687688571694849  / semi partial :  -0.7007991095104914  코로나와 상관있음
대구 평일 군집1 의료기관(70) -0.6451658789467998  / semi partial :  -0.6492426463652853  코로나와 상관있음
대구 평일 군집1 보건위생(71) -0.5827322140739886  / semi partial :  -0.564425120648835  코로나와 상관있음
대구 평일 군집1 요식업소(80) -0.6421058506373907  / semi partial :  -0.6073999796761396  코로나와 상관있음
---- 2군집 (주거상권) ----
['남산4동', '만촌3동', '범어3동', '범물1동', '범어4동']
대구 평일 군집2 레저업소(21) -0.40766511712345743  / semi partial :  -0.4331618332201549  코로나와 상관있음
대구 평일 군집2 의료기관(70) -0.5333979068058635  / semi partial :  -0.5117103636433199  코로나와 상관있음
대구 평일 군집2 요식업소(80) -0.5849895433755977  / semi 

### 카드 대구 휴일 (2020, 2019)

In [ ]:
# 2020년
# 각 cluster와 대구 카드 휴일 교집합 
daegu_weekend_seg1_list = list(set(daegu_cluster1).intersection(set(card_weekend_list)))
daegu_weekend_seg2_list = list(set(daegu_cluster2).intersection(set(card_weekend_list)))
daegu_weekend_seg3_list = list(set(daegu_cluster3).intersection(set(card_weekend_list)))
daegu_weekend_seg4_list = list(set(daegu_cluster4).intersection(set(card_weekend_list)))
print(daegu_weekend_seg1_list, '\n', daegu_weekend_seg2_list, '\n', daegu_weekend_seg3_list, '\n', daegu_weekend_seg4_list)

# 대구 휴일 각 군집 분리 데이터프레임 생성
card_daegu_weekend_seg1 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(daegu_weekend_seg1_list)]
card_daegu_weekend_seg2 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(daegu_weekend_seg2_list)] 
card_daegu_weekend_seg3 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(daegu_weekend_seg3_list)]
card_daegu_weekend_seg4 = card2020_weekend[card2020_weekend['HDONG_NM'].isin(daegu_weekend_seg4_list)]

print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)

# 휴일 각 군집별 일별, 업종별 USE_AMT 평균
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg1 = pd.DataFrame(card_daegu_weekend_seg1)

card_daegu_weekend_seg2 = card_daegu_weekend_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg2 = pd.DataFrame(card_daegu_weekend_seg2)

card_daegu_weekend_seg3 = card_daegu_weekend_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg3 = pd.DataFrame(card_daegu_weekend_seg3)

card_daegu_weekend_seg4 = card_daegu_weekend_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg4 = pd.DataFrame(card_daegu_weekend_seg4)

print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)

# 데이터 reset_index
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.reset_index()
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.reset_index()
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.reset_index()
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_daegu_weekend_seg1["MCT_CAT_CD"] = card_daegu_weekend_seg1["MCT_CAT_CD"].astype(str)
card_daegu_weekend_seg2["MCT_CAT_CD"] = card_daegu_weekend_seg2["MCT_CAT_CD"].astype(str)
card_daegu_weekend_seg3["MCT_CAT_CD"] = card_daegu_weekend_seg3["MCT_CAT_CD"].astype(str)
card_daegu_weekend_seg4["MCT_CAT_CD"] = card_daegu_weekend_seg4["MCT_CAT_CD"].astype(str)

print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)

['두산동', '범어1동', '중동', '대봉1동', '대신동', '범어2동', '성내2동'] 
 ['고산3동', '고산1동', '남산4동', '만촌3동', '범물1동', '범어4동'] 
 ['황금1동', '상동', '파동', '대봉2동'] 
 ['고산2동', '만촌2동', '성내3동', '만촌1동', '남산2동', '동인동', '황금2동', '남산1동']
(41824, 13) (29425, 13) (12234, 13) (34492, 13)
(836, 1) (701, 1) (590, 1) (818, 1)
(836, 3) (701, 3) (590, 3) (818, 3)


In [ ]:
# pivot table 로 펼침 (휴일)
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")

# 결측치 0
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.fillna(0)
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.fillna(0)
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.fillna(0)
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.fillna(0)

# 전치 뒤집기
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.T
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.T
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.T
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.T

print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_daegu_weekend_seg1_corr = pd.merge(holiday, card_daegu_weekend_seg1, on='STD_DD')
card_daegu_weekend_seg2_corr = pd.merge(holiday, card_daegu_weekend_seg2,on='STD_DD')
card_daegu_weekend_seg3_corr = pd.merge(holiday, card_daegu_weekend_seg3,on='STD_DD')
card_daegu_weekend_seg4_corr = pd.merge(holiday, card_daegu_weekend_seg4,on='STD_DD')

# covid와 merge
card_daegu_weekend_seg1_corr = pd.merge(covid, card_daegu_weekend_seg1_corr, on='STD_DD')
card_daegu_weekend_seg2_corr = pd.merge(covid, card_daegu_weekend_seg2_corr,on='STD_DD')
card_daegu_weekend_seg3_corr = pd.merge(covid, card_daegu_weekend_seg3_corr,on='STD_DD')
card_daegu_weekend_seg4_corr = pd.merge(covid, card_daegu_weekend_seg4_corr,on='STD_DD')

# card 23개 업종 pivot table + covid + holiday
print(card_daegu_weekend_seg1_corr.shape , card_daegu_weekend_seg2_corr.shape, card_daegu_weekend_seg3_corr.shape, card_daegu_weekend_seg4_corr.shape)

# copy
card_daegu_weekend_seg1_corr_2020 = card_daegu_weekend_seg1_corr.copy()
card_daegu_weekend_seg2_corr_2020 = card_daegu_weekend_seg2_corr.copy()
card_daegu_weekend_seg3_corr_2020 = card_daegu_weekend_seg3_corr.copy()
card_daegu_weekend_seg4_corr_2020 = card_daegu_weekend_seg4_corr.copy()

print(card_daegu_weekend_seg1_corr_2020.shape , card_daegu_weekend_seg2_corr_2020.shape, card_daegu_weekend_seg3_corr_2020.shape, card_daegu_weekend_seg4_corr_2020.shape)

(39, 23) (39, 23) (39, 22) (39, 23)
(39, 27) (39, 27) (39, 26) (39, 27)
(39, 27) (39, 27) (39, 26) (39, 27)


In [ ]:
# 2019년
# 각 cluster와 대구 카드 휴일 교집합 
daegu_weekend_seg1_list = list(set(daegu_cluster1).intersection(set(card_weekend_list)))
daegu_weekend_seg2_list = list(set(daegu_cluster2).intersection(set(card_weekend_list)))
daegu_weekend_seg3_list = list(set(daegu_cluster3).intersection(set(card_weekend_list)))
daegu_weekend_seg4_list = list(set(daegu_cluster4).intersection(set(card_weekend_list)))
print(daegu_weekend_seg1_list, '\n', daegu_weekend_seg2_list, '\n', daegu_weekend_seg3_list, '\n', daegu_weekend_seg4_list)

# 대구 휴일 각 군집 분리 데이터프레임 생성
card_daegu_weekend_seg1 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(daegu_weekend_seg1_list)]
card_daegu_weekend_seg2 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(daegu_weekend_seg2_list)] 
card_daegu_weekend_seg3 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(daegu_weekend_seg3_list)]
card_daegu_weekend_seg4 = card2019_weekend[card2019_weekend['HDONG_NM'].isin(daegu_weekend_seg4_list)]

print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)

# 휴일 각 군집별 일별, 업종별 USE_AMT 평균
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg1 = pd.DataFrame(card_daegu_weekend_seg1)

card_daegu_weekend_seg2 = card_daegu_weekend_seg2.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg2 = pd.DataFrame(card_daegu_weekend_seg2)

card_daegu_weekend_seg3 = card_daegu_weekend_seg3.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg3 = pd.DataFrame(card_daegu_weekend_seg3)

card_daegu_weekend_seg4 = card_daegu_weekend_seg4.groupby(['STD_DD','MCT_CAT_CD'])['USE_AMT'].agg('mean')
card_daegu_weekend_seg4 = pd.DataFrame(card_daegu_weekend_seg4)

print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)

# 데이터 reset_index
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.reset_index()
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.reset_index()
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.reset_index()
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.reset_index()

# 데이터 업종 컬럼, str 형으로 변환
card_daegu_weekend_seg1["MCT_CAT_CD"] = card_daegu_weekend_seg1["MCT_CAT_CD"].astype(str)
card_daegu_weekend_seg2["MCT_CAT_CD"] = card_daegu_weekend_seg2["MCT_CAT_CD"].astype(str)
card_daegu_weekend_seg3["MCT_CAT_CD"] = card_daegu_weekend_seg3["MCT_CAT_CD"].astype(str)
card_daegu_weekend_seg4["MCT_CAT_CD"] = card_daegu_weekend_seg4["MCT_CAT_CD"].astype(str)

print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)

['두산동', '범어1동', '중동', '대봉1동', '대신동', '범어2동', '성내2동'] 
 ['고산3동', '고산1동', '남산4동', '만촌3동', '범물1동', '범어4동'] 
 ['황금1동', '상동', '파동', '대봉2동'] 
 ['고산2동', '만촌2동', '성내3동', '만촌1동', '남산2동', '동인동', '황금2동', '남산1동']
(49571, 13) (33214, 13) (13897, 13) (39272, 13)
(848, 1) (686, 1) (611, 1) (836, 1)
(848, 3) (686, 3) (611, 3) (836, 3)


In [ ]:
# pivot table 로 펼침 (휴일)
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.pivot(index="MCT_CAT_CD", columns="STD_DD", values="USE_AMT")

# 결측치 0
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.fillna(0)
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.fillna(0)
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.fillna(0)
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.fillna(0)

# 전치 뒤집기
card_daegu_weekend_seg1 = card_daegu_weekend_seg1.T
card_daegu_weekend_seg2 = card_daegu_weekend_seg2.T
card_daegu_weekend_seg3 = card_daegu_weekend_seg3.T
card_daegu_weekend_seg4 = card_daegu_weekend_seg4.T

print(card_daegu_weekend_seg1.shape, card_daegu_weekend_seg2.shape, card_daegu_weekend_seg3.shape, card_daegu_weekend_seg4.shape)

# 휴일여부 (holiday) 와 merge
card_daegu_weekend_seg1_corr = pd.merge(holiday, card_daegu_weekend_seg1, on='STD_DD')
card_daegu_weekend_seg2_corr = pd.merge(holiday, card_daegu_weekend_seg2,on='STD_DD')
card_daegu_weekend_seg3_corr = pd.merge(holiday, card_daegu_weekend_seg3,on='STD_DD')
card_daegu_weekend_seg4_corr = pd.merge(holiday, card_daegu_weekend_seg4,on='STD_DD')

# covid와 merge
card_daegu_weekend_seg1_corr = pd.merge(covid, card_daegu_weekend_seg1_corr, on='STD_DD')
card_daegu_weekend_seg2_corr = pd.merge(covid, card_daegu_weekend_seg2_corr,on='STD_DD')
card_daegu_weekend_seg3_corr = pd.merge(covid, card_daegu_weekend_seg3_corr,on='STD_DD')
card_daegu_weekend_seg4_corr = pd.merge(covid, card_daegu_weekend_seg4_corr,on='STD_DD')

# card 23개 업종 pivot table + covid + holiday
print(card_daegu_weekend_seg1_corr.shape , card_daegu_weekend_seg2_corr.shape, card_daegu_weekend_seg3_corr.shape, card_daegu_weekend_seg4_corr.shape)

# copy
card_daegu_weekend_seg1_corr_2019 = card_daegu_weekend_seg1_corr.copy()
card_daegu_weekend_seg2_corr_2019 = card_daegu_weekend_seg2_corr.copy()
card_daegu_weekend_seg3_corr_2019 = card_daegu_weekend_seg3_corr.copy()
card_daegu_weekend_seg4_corr_2019 = card_daegu_weekend_seg4_corr.copy()

print(card_daegu_weekend_seg1_corr_2019.shape , card_daegu_weekend_seg2_corr_2019.shape, card_daegu_weekend_seg3_corr_2019.shape, card_daegu_weekend_seg4_corr_2019.shape)

(39, 23) (39, 22) (39, 22) (39, 23)
(39, 27) (39, 26) (39, 26) (39, 27)
(39, 27) (39, 26) (39, 26) (39, 27)


In [ ]:
# 최종 19, 20 합쳐진 카드 데이터 - 대구 휴일
card_daegu_weekend_seg1_corr_19_20 = pd.merge(card_daegu_weekend_seg1_corr_2019, card_daegu_weekend_seg1_corr_2020, on='STD_DD')
card_daegu_weekend_seg2_corr_19_20 = pd.merge(card_daegu_weekend_seg2_corr_2019, card_daegu_weekend_seg2_corr_2020, on='STD_DD')
card_daegu_weekend_seg3_corr_19_20 = pd.merge(card_daegu_weekend_seg3_corr_2019, card_daegu_weekend_seg3_corr_2020, on='STD_DD')
card_daegu_weekend_seg4_corr_19_20 = pd.merge(card_daegu_weekend_seg4_corr_2019, card_daegu_weekend_seg4_corr_2020, on='STD_DD')

print(card_daegu_weekend_seg1_corr_19_20.shape , card_daegu_weekend_seg2_corr_19_20.shape, card_daegu_weekend_seg3_corr_19_20.shape, card_daegu_weekend_seg4_corr_19_20.shape)
card_daegu_weekend_seg1_corr_19_20

(34, 53) (34, 52) (34, 51) (34, 53)


,STD_DD,confirmed_x,day_x,weekend_x,10_x,20_x,21_x,22_x,30_x,31_x,32_x,33_x,34_x,35_x,40_x,42_x,43_x,44_x,50_x,52_x,60_x,62_x,70_x,71_x,80_x,81_x,92_x,confirmed_y,day_y,weekend_y,10_y,20_y,21_y,22_y,30_y,31_y,32_y,33_y,34_y,35_y,40_y,42_y,43_y,44_y,50_y,52_y,60_y,62_y,70_y,71_y,80_y,81_y,92_y
0,20200201,4,5,1,781.671875,1982.947368,704.597403,892.706667,8147.666667,0.0,1819.529412,2572.336538,643.312500,8602.600000,12908.439490,2264.100000,717.692308,624.305556,360.461538,14021.150000,1231.142857,1032.413333,4623.535714,1105.137615,6369.613924,1236.085714,168.066667,4,5,1,1140.307692,570.296296,623.320988,486.782609,6288.666667,0.0,792.166667,2312.846154,1523.272727,1676.875000,8130.709091,1840.075758,656.923077,678.000000,428.857143,13046.527778,3591.333333,324.942857,3724.866667,781.520833,5107.000000,498.753521,131.384615
1,20200202,1,6,1,779.848485,1297.842105,487.957746,928.241935,0.000000,0.0,613.857143,2294.980198,763.250000,5375.076923,12607.364780,1385.020000,1301.307692,241.117647,201.368421,17342.400000,0.000000,76.295455,1002.791667,798.397849,5916.493976,1080.692857,58.000000,1,6,1,1457.972222,668.958333,399.368421,399.192982,5006.500000,0.0,458.666667,1977.000000,746.647059,3327.700000,7805.920732,1069.512195,246.000000,334.200000,268.240000,11438.218750,5270.000000,174.225000,735.285714,638.457831,3542.317365,474.720000,117.500000
2,20200208,0,5,1,875.222222,1012.608696,680.514706,874.625000,4165.750000,7029.0,1321.571429,2276.895238,1372.166667,3549.909091,10431.950311,2264.441176,1263.117647,503.166667,377.040000,10912.522727,9485.000000,411.160494,4767.985185,844.930000,5433.677019,591.237037,98.850000,0,5,1,1446.835616,2132.409091,768.975000,530.233333,10011.000000,0.0,805.571429,2209.070000,1237.466667,3042.714286,9477.689441,2039.870370,1266.000000,678.897436,585.054054,11532.268293,896.000000,364.281250,4000.817518,689.437500,5009.451807,518.939850,77.952381
3,20200209,0,6,1,858.210526,4598.400000,675.090909,969.534483,1282.000000,0.0,590.375000,2162.257732,1598.076923,5516.500000,8003.183544,2728.377049,1188.214286,1659.000000,308.153846,11906.354839,29755.000000,147.940000,1323.281250,737.250000,4218.896341,392.279070,108.000000,0,6,1,2873.308824,1826.391304,479.246575,350.979167,6186.000000,0.0,916.333333,2086.303030,1137.125000,3815.625000,7637.446541,1309.018519,743.400000,1460.500000,213.750000,13118.517241,4391.666667,127.136364,1010.887097,629.056818,3983.666667,397.173228,26.000000
4,20200215,0,5,1,1488.187500,735.411765,880.737500,883.315068,399.250000,0.0,1477.111111,2521.852941,1197.631579,5313.416667,11908.477987,2288.596774,1722.368421,789.974359,603.307692,13162.307692,1560.833333,389.637681,4358.022059,776.362745,5961.628931,663.466165,88.920000,0,5,1,2017.549296,1456.391304,765.573171,360.854545,6069.333333,0.0,2521.250000,2369.556701,1702.722222,1879.166667,9783.223602,2066.876923,1345.066667,993.891892,329.000000,14134.405405,9431.500000,322.253521,3914.613139,779.590909,6003.935673,776.292857,189.125000
5,20200216,0,6,1,945.048780,1718.636364,343.200000,762.774194,5778.666667,0.0,120.250000,2304.489362,1225.578947,5635.545455,9797.124224,805.789474,549.571429,186.142857,269.650000,12822.812500,0.000000,231.604651,831.785714,594.430380,5253.296774,398.155039,108.000000,0,6,1,1834.055556,1088.666667,323.907895,606.285714,4582.000000,0.0,153.000000,2027.135922,889.307692,2179.000000,9581.870370,948.403846,587.333333,1342.391304,344.608696,11997.800000,4321.400000,109.527778,708.810345,602.133333,4083.500000,395.950820,125.250000
6,20200222,100,5,1,2649.794872,892.875000,405.227273,923.305556,5203.750000,3273.0,710.588235,2382.590476,709.555556,4185.333333,11078.500000,2726.842857,1397.857143,732.578947,427.552632,14007.410256,5926.600000,538.948052,3937.528986,1023.350515,6732.522013,606.572464,132.166667,100,5,1,1405.166667,435.750000,335.312500,575.928571,3749.000000,377.0,142.000000,1403.206522,1512.600000,1537.125000,4230.083871,1227.045455,116.000000,386.000000,183.000000,4781.454545

In [ ]:
# 대구 휴일 seg1 cor_val이상 업종
print('대구 카드 휴일')

card_daegu_weekend_seg1_corr40_list = [] # 상관계수 cor_val 이상인 업종
card_daegu_weekend_seg2_corr40_list = [] # 상관계수 cor_val 이상인 업종
card_daegu_weekend_seg3_corr40_list = [] # 상관계수 cor_val 이상인 업종
card_daegu_weekend_seg4_corr40_list = [] # 상관계수 cor_val 이상인 업종

print('---- 1군집 (상업지역) ----')
print(daegu_weekend_seg1_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_daegu_weekend_seg1_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 휴일 군집1',type_list[i], card_daegu_weekend_seg1_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_daegu_weekend_seg1_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_daegu_weekend_seg1_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_daegu_weekend_seg1_corr40_list.append(type_list[i])
  except:
    pass

# 대구 휴일 seg2 cor_val이상 업종
print('---- 2군집 (주거상권) ----')
print(daegu_weekend_seg2_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_daegu_weekend_seg2_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 휴일 군집2',type_list[i], card_daegu_weekend_seg2_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_daegu_weekend_seg2_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_daegu_weekend_seg2_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_daegu_weekend_seg2_corr40_list.append(type_list[i])
        
  except:
    pass
print()

# 대구 휴일 seg3 cor_val이상 업종
print('---- 3군집 (주거지역) ----')
print(daegu_weekend_seg3_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_daegu_weekend_seg3_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 휴일 군집3',type_list[i], card_daegu_weekend_seg3_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_daegu_weekend_seg3_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_daegu_weekend_seg3_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_daegu_weekend_seg3_corr40_list.append(type_list[i])
  except:
    pass
print()

# 대구 휴일 seg4 cor_val이상 업종
print('---- 4군집 (교통중심) ----')
print(daegu_weekend_seg4_list)
for i in MCT_CAT_CD_list:
  try:
    if abs(card_daegu_weekend_seg4_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 휴일 군집4',type_list[i], card_daegu_weekend_seg4_corr_19_20[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=card_daegu_weekend_seg4_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=card_daegu_weekend_seg4_corr_19_20, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        card_daegu_weekend_seg4_corr40_list.append(type_list[i])
  except:
    pass       
print()


대구 카드 휴일
---- 1군집 (상업지역) ----
['두산동', '범어1동', '중동', '대봉1동', '대신동', '범어2동', '성내2동']
대구 휴일 군집1 레저업소(21) -0.4601621516315583  / semi partial :  -0.34381971014348245 코로나와 상관없음. 우연에 의함
대구 휴일 군집1 문화취미(22) -0.36255167599691906  / semi partial :  -0.35285031755145835  코로나와 상관있음
대구 휴일 군집1 연료판매(33) -0.598079669603421  / semi partial :  -0.44787907493248746  코로나와 상관있음
대구 휴일 군집1 유통업(40) -0.6623821816654101  / semi partial :  -0.6766530884694459  코로나와 상관있음
대구 휴일 군집1 서적문구(50) -0.36243697767060984  / semi partial :  -0.35151602280697364  코로나와 상관있음
대구 휴일 군집1 사무통신(52) -0.7184768343332679  / semi partial :  -0.6560792128982095  코로나와 상관있음
대구 휴일 군집1 요식업소(80) -0.6564893360199694  / semi partial :  -0.6245824320972003  코로나와 상관있음
---- 2군집 (주거상권) ----
['고산3동', '고산1동', '남산4동', '만촌3동', '범물1동', '범어4동']
대구 휴일 군집2 의료기관(70) -0.44372102570922195  / semi partial :  -0.33050209478064674 코로나와 상관없음. 우연에 의함
대구 휴일 군집2 보건위생(71) -0.43858463891837907  / semi partial :  -0.45350896642879945  코로나와 상관있음
대구 휴일 군집2 요식업소(80) -0.63

## 4-2. 군집별 편의점 업종별 상관분석
- 서울 평일 / 서울 휴일 / 대구 평일 / 대구 휴일

### 데이터 Cleansing & 전처리
- 편의점데이터 2020 구분
  - gs2020
- 편의점 평일, 휴일 데이터 구분
  - gs2020_weekday, gs2020_weekend

In [ ]:
# gs row 전처리
gs_row = gs_row.rename(columns={'OPER_DT':'STD_DD', 'ADMD':'HDONG_NM'})

# gs2020 데이터
gs2020 = gs_row[gs_row['STD_DD']>20191231]
gs2020.reset_index(inplace=True, drop=True)
del gs2020['PVN']; del gs2020['BOR']; #del gs2020['AMT_IND']

# gs2020
gs2020 = pd.merge(holiday, gs2020, on='STD_DD')
gs2020 = pd.merge(covid, gs2020, on='STD_DD' )

# 카테고리 비중이 문자열로 된 것을 변환
index_list=['LCLS_10_P', 'LCLS_20_P', 'LCLS_30_P', 'LCLS_40_P', 'LCLS_50_P', 'LCLS_60_P', 'LCLS_70_P','LCLS_80_P']

# 각 동의 전체 매출에 대한 카테고리별 퍼센트로 나와있어서 이것을 전체 매출과 곱하여 카테고리별 절대 지수를 만듬 -> 매출지수
for i in index_list:
  gs2020['mul_'+i]=gs2020['AMT_IND']*gs2020[i]*100
  del gs2020[i]

# 평일 휴일 나누기
gs2020_weekday = gs2020[gs2020['weekend']==0]

# 휴일 데이터
gs2020_weekend = gs2020[gs2020['weekend'] == 1]

gs2020_weekday.reset_index(inplace=True, drop=True)
gs2020_weekend.reset_index(inplace=True, drop=True)

print(gs2020.shape, gs2020_weekday.shape, gs2020_weekend.shape)

# 업종 list
LCLS_list = gs2020.columns[6:]
print(len(LCLS_list),LCLS_list)

(7260, 14) (4920, 14) (2340, 14)
8 Index(['mul_LCLS_10_P', 'mul_LCLS_20_P', 'mul_LCLS_30_P', 'mul_LCLS_40_P',
       'mul_LCLS_50_P', 'mul_LCLS_60_P', 'mul_LCLS_70_P', 'mul_LCLS_80_P'],
      dtype='object')


In [ ]:
# gs 2019 처리
gs_row2019_ls = gs_row['STD_DD']<20191231
gs2019 = gs_row[gs_row2019_ls]
gs2019.reset_index(inplace=True, drop=True)
del gs2019['PVN']; del gs2019['BOR']; #del gs2020['AMT_IND']

# gs2020
gs2019 = pd.merge(holiday, gs2019, on='STD_DD')

# 카테고리 비중이 문자열로 된 것을 변환
index_list=['LCLS_10_P', 'LCLS_20_P', 'LCLS_30_P', 'LCLS_40_P', 'LCLS_50_P', 'LCLS_60_P', 'LCLS_70_P','LCLS_80_P']

# 각 동의 전체 매출에 대한 카테고리별 퍼센트로 나와있어서 이것을 전체 매출과 곱하여 카테고리별 절대 지수를 만듬  
for i in index_list:
  gs2019['mul_'+i]=gs2019['AMT_IND']*gs2019[i]*100
  del gs2019[i]

# gs2019 데이터 2020으로 날짜 맞춰주기 timedelta 이용
gs2019['STD_DD'] = gs2019['STD_DD'].astype(str)
gs2019['STD_DD'] = pd.to_datetime(gs2019['STD_DD'])
gs2019['STD_DD'] = pd.DatetimeIndex(gs2019['STD_DD']) + timedelta(weeks=52)

gs2019['STD_DD'] = gs2019['STD_DD'].astype(str)
gs2019["STD_DD"] = gs2019["STD_DD"].str.replace("-","")
gs2019["STD_DD"] = gs2019["STD_DD"].astype(int)

# gs 19 20 날짜 합친 것.
gs_19_20 = pd.merge(gs2019, gs2020, on=['STD_DD', 'HDONG_NM'])
gs_19_20 = pd.merge(gs_19_20, covid, on = ['STD_DD'])

# 평일 휴일 나누기
gs_19_20_weekday = gs_19_20[gs_19_20['weekend_x']==0]
gs_19_20_weekend = gs_19_20[gs_19_20['weekend_x']==1]

gs_19_20_weekday.reset_index(inplace=True, drop=True)
gs_19_20_weekend.reset_index(inplace=True, drop=True)

print(gs_19_20.shape, gs_19_20_weekday.shape, gs_19_20_weekend.shape)
gs_19_20

(7140, 26) (4800, 26) (2340, 26)


,STD_DD,day_x,weekend_x,HDONG_NM,AMT_IND_x,mul_LCLS_10_P_x,mul_LCLS_20_P_x,mul_LCLS_30_P_x,mul_LCLS_40_P_x,mul_LCLS_50_P_x,mul_LCLS_60_P_x,mul_LCLS_70_P_x,mul_LCLS_80_P_x,confirmed_x,day_y,weekend_y,AMT_IND_y,mul_LCLS_10_P_y,mul_LCLS_20_P_y,mul_LCLS_30_P_y,mul_LCLS_40_P_y,mul_LCLS_50_P_y,mul_LCLS_60_P_y,mul_LCLS_70_P_y,mul_LCLS_80_P_y,confirmed_y
0,20200201,5,1,고산1동,0.940026,27.677739,18.980567,36.314502,2.729587,6.631926,0.609912,0.669998,0.388402,4,5,1,0.854239,22.660598,19.670760,31.086279,3.054371,7.555833,0.494051,0.569419,0.332563,4
1,20200201,5,1,고산2동,1.172479,44.389652,23.625214,35.771146,1.435410,7.662538,1.153544,1.887421,1.322987,4,5,1,1.080543,35.531166,27.467702,36.191390,1.101706,5.976038,0.000000,1.178058,0.608279,4
2,20200201,5,1,고산3동,0.895912,34.374076,18.128174,28.175804,0.686017,4.996726,0.881955,1.806460,0.541952,4,5,1,0.759088,23.650104,16.925769,28.587669,0.990256,4.197883,0.000000,1.217369,0.339751,4
3,20200201,5,1,두산동,1.092902,24.283429,24.905694,45.970941,2.457469,9.456863,0.682309,0.593916,0.939606,4,5,1,0.911892,17.675688,20.850350,31.331062,3.068180,14.231147,0.090823,2.876597,1.065312,4
4,20200201,5,1,만촌1동,1.253741,50.045001,25.991590,40.460214,3.107759,4.421891,0.898299,0.206108,0.243204,4,5,1,1.124104,34.438142,28.609536,35.419524,2.716673,7.735932,1.173468,0.539972,1.777157,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7135,20200529,4,0,중림동,1.104929,33.587966,24.480168,44.267867,2.612954,4.576105,0.000000,0.866170,0.101623,79,4,0,1.156155,37.579270,25.444363,40.216927,2.546906,6.993686,2.032528,0.801787,0.000000,79
7136,20200529,4,0,청구동,1.464850,43.946314,31.438228,57.398650,6.060868,4.589281,1.883208,0.500717,0.667784,79,4,0,1.563844,43.949677,39.068269,58.904006,6.319895,7.282288,0.088826,0.029429,0.741977,79
7137,20200529,4,0,필동,1.734506,53.086083,34.561248,66.755000,5.836981,9.891391,0.948948,1.518038,0.852941,79,4,0,1.285247,39.146678,26.147883,47.763488,5.092193,8.392937,0.697045,1.240833,0.043642,79
7138,20200529,4,0,황학동,1.163772,33.133533,24.697378,45.401658,3.784102,6.149560,1.245609,0.848000,1.117311,79,4,0,1.155756,33.161728,26.420707,45.185221,3.688446,6.616800,0.149707,0.000000,0.353007,79


### 편의점 서울 평일

In [ ]:
# 각 cluster와 서울 편의점 평일 교집합 
seoul_weekday_seg1_list = list(set(seoul_cluster1).intersection(set(gs_weekday_list)))
seoul_weekday_seg2_list = list(set(seoul_cluster2).intersection(set(gs_weekday_list)))
seoul_weekday_seg3_list = list(set(seoul_cluster3).intersection(set(gs_weekday_list)))
seoul_weekday_seg4_list = list(set(seoul_cluster4).intersection(set(gs_weekday_list)))
print(seoul_weekday_seg1_list, '\n', seoul_weekday_seg2_list, '\n', seoul_weekday_seg3_list, '\n', seoul_weekday_seg4_list)

# 서울 평일 각 군집 로 분리 데이터프레임 생성
gs_seoul_weekday_seg1 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(seoul_weekday_seg1_list)]
gs_seoul_weekday_seg2 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(seoul_weekday_seg2_list)] 
gs_seoul_weekday_seg3 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(seoul_weekday_seg3_list)]
gs_seoul_weekday_seg4 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(seoul_weekday_seg4_list)]

print(gs_seoul_weekday_seg1.shape, gs_seoul_weekday_seg2.shape, gs_seoul_weekday_seg3.shape, gs_seoul_weekday_seg4.shape)

# 평일 각 군집 별 일별 업종별 
gs_seoul_weekday_seg1_corr = gs_seoul_weekday_seg1.groupby('STD_DD').agg('mean'); gs_seoul_weekday_seg1_corr.reset_index(level='STD_DD', inplace=True)
gs_seoul_weekday_seg2_corr = gs_seoul_weekday_seg2.groupby('STD_DD').agg('mean'); gs_seoul_weekday_seg2_corr.reset_index(level='STD_DD', inplace=True)
gs_seoul_weekday_seg3_corr = gs_seoul_weekday_seg3.groupby('STD_DD').agg('mean'); gs_seoul_weekday_seg3_corr.reset_index(level='STD_DD', inplace=True)
gs_seoul_weekday_seg4_corr = gs_seoul_weekday_seg4.groupby('STD_DD').agg('mean'); gs_seoul_weekday_seg4_corr.reset_index(level='STD_DD', inplace=True)

print(gs_seoul_weekday_seg1_corr.shape, gs_seoul_weekday_seg2_corr.shape, gs_seoul_weekday_seg3_corr.shape, gs_seoul_weekday_seg4_corr.shape)

['필동', '회현동', '소공동', '상계2동'] 
 ['상계9동', '하계1동', '중계1동', '중림동', '월계1동', '상계1동'] 
 ['청구동', '상계3.4동', '상계5동', '월계3동', '상계10동'] 
 ['공릉2동', '을지로동', '신당동', '상계6.7동', '공릉1동', '신당5동', '약수동', '황학동', '장충동']
(320, 26) (480, 26) (400, 26) (720, 26)
(80, 25) (80, 25) (80, 25) (80, 25)


In [ ]:
# 서울 평일 seg1 cor_val이상 업종
print('서울 편의점 평일')

gs_seoul_weekday_seg1_corr40_list = [] # 상관계수 cor_val 이상인 업종
gs_seoul_weekday_seg2_corr40_list = [] # 상관계수 cor_val 이상인 업종
gs_seoul_weekday_seg3_corr40_list = [] # 상관계수 cor_val 이상인 업종
gs_seoul_weekday_seg4_corr40_list = [] # 상관계수 cor_val 이상인 업종

print('---- 1군집 (상업지역) ----')
print(seoul_weekday_seg1_list)
for i in LCLS_list:
  try:
    if abs(gs_seoul_weekday_seg1_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 평일 군집1',circulation_list[i], gs_seoul_weekday_seg1_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_seoul_weekday_seg1_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_seoul_weekday_seg1_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_seoul_weekday_seg1_corr40_list.append(circulation_list[i])
  except:
    pass

# 서울 평일 seg2 cor_val이상 업종
print('---- 2군집 (주거상권) ----')
print(seoul_weekday_seg2_list)
for i in LCLS_list:
  try:
    if abs(gs_seoul_weekday_seg2_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 평일 군집2',circulation_list[i], gs_seoul_weekday_seg2_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_seoul_weekday_seg2_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_seoul_weekday_seg2_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_seoul_weekday_seg2_corr40_list.append(circulation_list[i])
        
  except:
    pass
print()

# 서울 평일 seg3 cor_val이상 업종
print('---- 3군집 (주거지역) ----')
print(seoul_weekday_seg3_list)
for i in LCLS_list:
  try:
    if abs(gs_seoul_weekday_seg3_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 평일 군집3',circulation_list[i], gs_seoul_weekday_seg3_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_seoul_weekday_seg3_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_seoul_weekday_seg3_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_seoul_weekday_seg3_corr40_list.append(circulation_list[i])
  except:
    pass
print()

# 서울 평일 seg4 cor_val이상 업종
print('---- 4군집 (교통중심) ----')
print(seoul_weekday_seg4_list)
for i in LCLS_list:
  try:
    if abs(gs_seoul_weekday_seg4_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 평일 군집4',circulation_list[i], gs_seoul_weekday_seg4_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_seoul_weekday_seg4_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_seoul_weekday_seg4_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_seoul_weekday_seg4_corr40_list.append(circulation_list[i])
  except:
    pass       
print()


서울 편의점 평일
---- 1군집 (상업지역) ----
['필동', '회현동', '소공동', '상계2동']
서울 평일 군집1 마실거리 -0.4446249611693998  / semi partial :  -0.42574744996426506  코로나와 상관있음
---- 2군집 (주거상권) ----
['상계9동', '하계1동', '중계1동', '중림동', '월계1동', '상계1동']
서울 평일 군집2 식사 -0.39450150891946817  / semi partial :  -0.4423504868655506  코로나와 상관있음
서울 평일 군집2 마실거리 -0.44938656654263015  / semi partial :  -0.2849720486014809 코로나와 상관없음. 우연에 의함

---- 3군집 (주거지역) ----
['청구동', '상계3.4동', '상계5동', '월계3동', '상계10동']
서울 평일 군집3 마실거리 -0.37575653164629685  / semi partial :  -0.1903017780060503 코로나와 상관없음. 우연에 의함

---- 4군집 (교통중심) ----
['공릉2동', '을지로동', '신당동', '상계6.7동', '공릉1동', '신당5동', '약수동', '황학동', '장충동']
서울 평일 군집4 마실거리 -0.4737568961640501  / semi partial :  -0.31601253187456907 코로나와 상관없음. 우연에 의함



### 편의점 서울 휴일

In [ ]:
# 각 cluster와 서울 편의점 휴일 교집합 
seoul_weekend_seg1_list = list(set(seoul_cluster1).intersection(set(gs_weekend_list)))
seoul_weekend_seg2_list = list(set(seoul_cluster2).intersection(set(gs_weekend_list)))
seoul_weekend_seg3_list = list(set(seoul_cluster3).intersection(set(gs_weekend_list)))
seoul_weekend_seg4_list = list(set(seoul_cluster4).intersection(set(gs_weekend_list)))
print(seoul_weekend_seg1_list, '\n', seoul_weekend_seg2_list, '\n', seoul_weekend_seg3_list, '\n', seoul_weekend_seg4_list)

# 서울 휴일 각 군집 로 분리 데이터프레임 생성
gs_seoul_weekend_seg1 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(seoul_weekend_seg1_list)]
gs_seoul_weekend_seg2 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(seoul_weekend_seg2_list)] 
gs_seoul_weekend_seg3 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(seoul_weekend_seg3_list)]
gs_seoul_weekend_seg4 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(seoul_weekend_seg4_list)]

print(gs_seoul_weekend_seg1.shape, gs_seoul_weekend_seg2.shape, gs_seoul_weekend_seg3.shape, gs_seoul_weekend_seg4.shape)

# 휴일 각 군집 별 일별 업종별 
gs_seoul_weekend_seg1_corr = gs_seoul_weekend_seg1.groupby('STD_DD').agg('mean'); gs_seoul_weekend_seg1_corr.reset_index(level='STD_DD', inplace=True)
gs_seoul_weekend_seg2_corr = gs_seoul_weekend_seg2.groupby('STD_DD').agg('mean'); gs_seoul_weekend_seg2_corr.reset_index(level='STD_DD', inplace=True)
gs_seoul_weekend_seg3_corr = gs_seoul_weekend_seg3.groupby('STD_DD').agg('mean'); gs_seoul_weekend_seg3_corr.reset_index(level='STD_DD', inplace=True)
gs_seoul_weekend_seg4_corr = gs_seoul_weekend_seg4.groupby('STD_DD').agg('mean'); gs_seoul_weekend_seg4_corr.reset_index(level='STD_DD', inplace=True)

print(gs_seoul_weekend_seg1_corr.shape, gs_seoul_weekend_seg2_corr.shape, gs_seoul_weekend_seg3_corr.shape, gs_seoul_weekend_seg4_corr.shape)

['소공동', '상계2동', '명동', '회현동', '필동'] 
 ['상계9동', '하계1동', '중계1동', '중계본동', '중계4동', '중림동', '월계1동', '상계1동'] 
 ['청구동', '상계5동', '하계2동', '월계3동', '상계10동', '월계2동'] 
 ['공릉2동', '광희동', '을지로동', '신당동', '상계6.7동', '공릉1동', '신당5동', '약수동', '황학동', '장충동']
(195, 26) (312, 26) (234, 26) (390, 26)
(39, 25) (39, 25) (39, 25) (39, 25)


In [ ]:
# 서울 휴일 seg1 cor_val이상 업종
print('서울 편의점 휴일')

gs_seoul_weekend_seg1_corr40_list = [] # 상관계수 cor_val 이상인 업종
gs_seoul_weekend_seg2_corr40_list = [] # 상관계수 cor_val 이상인 업종
gs_seoul_weekend_seg3_corr40_list = [] # 상관계수 cor_val 이상인 업종
gs_seoul_weekend_seg4_corr40_list = [] # 상관계수 cor_val 이상인 업종

print('---- 1군집 (상업지역) ----')
print(seoul_weekend_seg1_list)
for i in LCLS_list:
  try:
    if abs(gs_seoul_weekend_seg1_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=0.35:
      print('서울 휴일 군집1',circulation_list[i], gs_seoul_weekend_seg1_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_seoul_weekend_seg1_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_seoul_weekend_seg1_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_seoul_weekend_seg1_corr40_list.append(circulation_list[i])
  except:
    pass

# 서울 휴일 seg2 cor_val이상 업종
print('---- 2군집 (주거상권) ----')
print(seoul_weekend_seg2_list)
for i in LCLS_list:
  try:
    if abs(gs_seoul_weekend_seg2_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 휴일 군집2',circulation_list[i], gs_seoul_weekend_seg2_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_seoul_weekend_seg2_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_seoul_weekend_seg2_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_seoul_weekend_seg2_corr40_list.append(circulation_list[i])
        
  except:
    pass
print()

# 서울 휴일 seg3 cor_val이상 업종
print('---- 3군집 (주거지역) ----')
print(seoul_weekend_seg3_list)
for i in LCLS_list:
  try:
    if abs(gs_seoul_weekend_seg3_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 휴일 군집3',circulation_list[i], gs_seoul_weekend_seg3_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_seoul_weekend_seg3_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_seoul_weekend_seg3_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_seoul_weekend_seg3_corr40_list.append(circulation_list[i])
  except:
    pass
print()

# 서울 휴일 seg4 cor_val이상 업종
print('---- 4군집 (교통중심) ----')
print(seoul_weekend_seg4_list)
for i in LCLS_list:
  try:
    if abs(gs_seoul_weekend_seg4_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('서울 휴일 군집4',circulation_list[i], gs_seoul_weekend_seg4_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_seoul_weekend_seg4_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_seoul_weekend_seg4_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_seoul_weekend_seg4_corr40_list.append(circulation_list[i])
  except:
    pass       
print()


서울 편의점 휴일
---- 1군집 (상업지역) ----
['소공동', '상계2동', '명동', '회현동', '필동']
서울 휴일 군집1 취미&여가활동_비중 -0.3710357760129984  / semi partial :  -0.37397040229972617  코로나와 상관있음
서울 휴일 군집1 임신/육아_비중 -0.4676108978747232  / semi partial :  -0.4984020449649305  코로나와 상관있음
---- 2군집 (주거상권) ----
['상계9동', '하계1동', '중계1동', '중계본동', '중계4동', '중림동', '월계1동', '상계1동']
서울 휴일 군집2 식사 -0.3619418795263827  / semi partial :  -0.36096716808438156  코로나와 상관있음
서울 휴일 군집2 마실거리 -0.41472806355447234  / semi partial :  -0.3374184260777765 코로나와 상관없음. 우연에 의함
서울 휴일 군집2 임신/육아_비중 -0.41412587745303253  / semi partial :  -0.422362994249429  코로나와 상관있음

---- 3군집 (주거지역) ----
['청구동', '상계5동', '하계2동', '월계3동', '상계10동', '월계2동']

---- 4군집 (교통중심) ----
['공릉2동', '광희동', '을지로동', '신당동', '상계6.7동', '공릉1동', '신당5동', '약수동', '황학동', '장충동']
서울 휴일 군집4 마실거리 -0.4133271115844437  / semi partial :  -0.3838340068090202  코로나와 상관있음
서울 휴일 군집4 임신/육아_비중 -0.4851341627442781  / semi partial :  -0.41102247438738154  코로나와 상관있음



### 편의점 대구 평일

In [ ]:
# 각 cluster와 대구 편의점 평일 교집합 
daegu_weekday_seg1_list = list(set(daegu_cluster1).intersection(set(gs_weekday_list)))
daegu_weekday_seg2_list = list(set(daegu_cluster2).intersection(set(gs_weekday_list)))
daegu_weekday_seg3_list = list(set(daegu_cluster3).intersection(set(gs_weekday_list)))
daegu_weekday_seg4_list = list(set(daegu_cluster4).intersection(set(gs_weekday_list)))
print(daegu_weekday_seg1_list, '\n', daegu_weekday_seg2_list, '\n', daegu_weekday_seg3_list, '\n', daegu_weekday_seg4_list)

# 대구 평일 각 군집 로 분리 데이터프레임 생성
gs_daegu_weekday_seg1 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(daegu_weekday_seg1_list)]
gs_daegu_weekday_seg2 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(daegu_weekday_seg2_list)] 
gs_daegu_weekday_seg3 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(daegu_weekday_seg3_list)]
gs_daegu_weekday_seg4 = gs_19_20_weekday[gs_19_20_weekday['HDONG_NM'].isin(daegu_weekday_seg4_list)]

print(gs_daegu_weekday_seg1.shape, gs_daegu_weekday_seg2.shape, gs_daegu_weekday_seg3.shape, gs_daegu_weekday_seg4.shape)

# 휴일 각 군집 별 일별 업종별 
gs_daegu_weekday_seg1_corr = gs_daegu_weekday_seg1.groupby('STD_DD').agg('mean'); gs_daegu_weekday_seg1_corr.reset_index(level='STD_DD', inplace=True)
gs_daegu_weekday_seg2_corr = gs_daegu_weekday_seg2.groupby('STD_DD').agg('mean'); gs_daegu_weekday_seg2_corr.reset_index(level='STD_DD', inplace=True)
gs_daegu_weekday_seg3_corr = gs_daegu_weekday_seg3.groupby('STD_DD').agg('mean'); gs_daegu_weekday_seg3_corr.reset_index(level='STD_DD', inplace=True)
gs_daegu_weekday_seg4_corr = gs_daegu_weekday_seg4.groupby('STD_DD').agg('mean'); gs_daegu_weekday_seg4_corr.reset_index(level='STD_DD', inplace=True)

print(gs_daegu_weekday_seg1_corr.shape, gs_daegu_weekday_seg2_corr.shape, gs_daegu_weekday_seg3_corr.shape, gs_daegu_weekday_seg4_corr.shape)

['두산동', '범어1동', '성내1동', '중동', '범어2동', '삼덕동'] 
 ['고산3동', '고산1동', '남산4동', '만촌3동', '범어3동', '범물1동', '범어4동'] 
 ['황금1동', '수성4가동', '수성1가동'] 
 ['고산2동', '성내3동', '동인동', '황금2동', '남산1동']
(480, 26) (560, 26) (240, 26) (400, 26)
(80, 25) (80, 25) (80, 25) (80, 25)


In [ ]:
# 대구 평일 seg1 cor_val이상 업종
print('대구 편의점 평일')

gs_daegu_weekday_seg1_corr40_list = [] # 상관계수 cor_val 이상인 업종
gs_daegu_weekday_seg2_corr40_list = [] # 상관계수 cor_val 이상인 업종
gs_daegu_weekday_seg3_corr40_list = [] # 상관계수 cor_val 이상인 업종
gs_daegu_weekday_seg4_corr40_list = [] # 상관계수 cor_val 이상인 업종

print('---- 1군집 (상업지역) ----')
print(daegu_weekday_seg1_list)
for i in LCLS_list:
  try:
    if abs(gs_daegu_weekday_seg1_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 평일 군집1',circulation_list[i], gs_daegu_weekday_seg1_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_daegu_weekday_seg1_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_daegu_weekday_seg1_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_daegu_weekday_seg1_corr40_list.append(circulation_list[i])
  except:
    pass

# 대구 평일 seg2 cor_val이상 업종
print('---- 2군집 (주거상권) ----')
print(daegu_weekday_seg2_list)
for i in LCLS_list:
  try:
    if abs(gs_daegu_weekday_seg2_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 평일 군집2',circulation_list[i], gs_daegu_weekday_seg2_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_daegu_weekday_seg2_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_daegu_weekday_seg2_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_daegu_weekday_seg2_corr40_list.append(circulation_list[i])
        
  except:
    pass
print()

# 대구 평일 seg3 cor_val이상 업종
print('---- 3군집 (주거지역) ----')
print(daegu_weekday_seg3_list)
for i in LCLS_list:
  try:
    if abs(gs_daegu_weekday_seg3_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 평일 군집3',circulation_list[i], gs_daegu_weekday_seg3_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_daegu_weekday_seg3_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_daegu_weekday_seg3_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_daegu_weekday_seg3_corr40_list.append(circulation_list[i])
  except:
    pass
print()

# 대구 평일 seg4 cor_val이상 업종
print('---- 4군집 (교통중심) ----')
print(daegu_weekday_seg4_list)
for i in LCLS_list:
  try:
    if abs(gs_daegu_weekday_seg4_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 평일 군집4',circulation_list[i], gs_daegu_weekday_seg4_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_daegu_weekday_seg4_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_daegu_weekday_seg4_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_daegu_weekday_seg4_corr40_list.append(circulation_list[i])
  except:
    pass       
print()


대구 편의점 평일
---- 1군집 (상업지역) ----
['두산동', '범어1동', '성내1동', '중동', '범어2동', '삼덕동']
대구 평일 군집1 식사 -0.4367805593005405  / semi partial :  -0.42032213331359636  코로나와 상관있음
대구 평일 군집1 마실거리 -0.6139672588756967  / semi partial :  -0.5257165168246622  코로나와 상관있음
대구 평일 군집1 홈&리빙 -0.36403024267536255  / semi partial :  -0.3613977965769859  코로나와 상관있음
---- 2군집 (주거상권) ----
['고산3동', '고산1동', '남산4동', '만촌3동', '범어3동', '범물1동', '범어4동']
대구 평일 군집2 마실거리 -0.5927402769062546  / semi partial :  -0.4062430756835629  코로나와 상관있음

---- 3군집 (주거지역) ----
['황금1동', '수성4가동', '수성1가동']
대구 평일 군집3 마실거리 -0.49381566385873554  / semi partial :  -0.24041474424374418 코로나와 상관없음. 우연에 의함

---- 4군집 (교통중심) ----
['고산2동', '성내3동', '동인동', '황금2동', '남산1동']
대구 평일 군집4 마실거리 -0.5503483789373134  / semi partial :  -0.3690536148915362  코로나와 상관있음



### 편의점 대구 휴일

In [ ]:
# 각 cluster와 대구 편의점 휴일 교집합 
daegu_weekend_seg1_list = list(set(daegu_cluster1).intersection(set(gs_weekend_list)))
daegu_weekend_seg2_list = list(set(daegu_cluster2).intersection(set(gs_weekend_list)))
daegu_weekend_seg3_list = list(set(daegu_cluster3).intersection(set(gs_weekend_list)))
daegu_weekend_seg4_list = list(set(daegu_cluster4).intersection(set(gs_weekend_list)))
print(daegu_weekend_seg1_list, '\n', daegu_weekend_seg2_list, '\n', daegu_weekend_seg3_list, '\n', daegu_weekend_seg4_list)

# 대구 휴일 각 군집 로 분리 데이터프레임 생성
gs_daegu_weekend_seg1 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(daegu_weekend_seg1_list)]
gs_daegu_weekend_seg2 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(daegu_weekend_seg2_list)] 
gs_daegu_weekend_seg3 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(daegu_weekend_seg3_list)]
gs_daegu_weekend_seg4 = gs_19_20_weekend[gs_19_20_weekend['HDONG_NM'].isin(daegu_weekend_seg4_list)]

print(gs_daegu_weekend_seg1.shape, gs_daegu_weekend_seg2.shape, gs_daegu_weekend_seg3.shape, gs_daegu_weekend_seg4.shape)

# 휴일 각 군집 별 일별 업종별 
gs_daegu_weekend_seg1_corr = gs_daegu_weekend_seg1.groupby('STD_DD').agg('mean'); gs_daegu_weekend_seg1_corr.reset_index(level='STD_DD', inplace=True)
gs_daegu_weekend_seg2_corr = gs_daegu_weekend_seg2.groupby('STD_DD').agg('mean'); gs_daegu_weekend_seg2_corr.reset_index(level='STD_DD', inplace=True)
gs_daegu_weekend_seg3_corr = gs_daegu_weekend_seg3.groupby('STD_DD').agg('mean'); gs_daegu_weekend_seg3_corr.reset_index(level='STD_DD', inplace=True)
gs_daegu_weekend_seg4_corr = gs_daegu_weekend_seg4.groupby('STD_DD').agg('mean'); gs_daegu_weekend_seg4_corr.reset_index(level='STD_DD', inplace=True)

print(gs_daegu_weekend_seg1_corr.shape, gs_daegu_weekend_seg2_corr.shape, gs_daegu_weekend_seg3_corr.shape, gs_daegu_weekend_seg4_corr.shape)

['두산동', '범어1동', '성내1동', '중동', '범어2동', '성내2동', '삼덕동'] 
 ['고산3동', '고산1동', '남산4동', '만촌3동', '범어3동', '범물1동', '범어4동'] 
 ['황금1동', '수성4가동', '수성1가동'] 
 ['고산2동', '성내3동', '동인동', '만촌1동', '수성2.3가동', '남산1동']
(273, 26) (273, 26) (117, 26) (234, 26)
(39, 25) (39, 25) (39, 25) (39, 25)


In [ ]:
# 대구 휴일 seg1 cor_val이상 업종
print('대구 편의점 휴일')

gs_daegu_weekend_seg1_corr40_list = [] # 상관계수 cor_val 이상인 업종
gs_daegu_weekend_seg2_corr40_list = [] # 상관계수 cor_val 이상인 업종
gs_daegu_weekend_seg3_corr40_list = [] # 상관계수 cor_val 이상인 업종
gs_daegu_weekend_seg4_corr40_list = [] # 상관계수 cor_val 이상인 업종

print('---- 1군집 (상업지역) ----')
print(daegu_weekend_seg1_list)
for i in LCLS_list:
  try:
    if abs(gs_daegu_weekend_seg1_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 휴일 군집1',circulation_list[i], gs_daegu_weekend_seg1_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_daegu_weekend_seg1_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_daegu_weekend_seg1_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_daegu_weekend_seg1_corr40_list.append(circulation_list[i])
  except:
    pass

# 대구 휴일 seg2 cor_val이상 업종
print('---- 2군집 (주거상권) ----')
print(daegu_weekend_seg2_list)
for i in LCLS_list:
  try:
    if abs(gs_daegu_weekend_seg2_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 휴일 군집2',circulation_list[i], gs_daegu_weekend_seg2_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_daegu_weekend_seg2_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_daegu_weekend_seg2_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_daegu_weekend_seg2_corr40_list.append(circulation_list[i])
        
  except:
    pass
print()

# 대구 휴일 seg3 cor_val이상 업종
print('---- 3군집 (주거지역) ----')
print(daegu_weekend_seg3_list)
for i in LCLS_list:
  try:
    if abs(gs_daegu_weekend_seg3_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 휴일 군집3',circulation_list[i], gs_daegu_weekend_seg3_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_daegu_weekend_seg3_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_daegu_weekend_seg3_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_daegu_weekend_seg3_corr40_list.append(circulation_list[i])
  except:
    pass
print()

# 대구 휴일 seg4 cor_val이상 업종
print('---- 4군집 (교통중심) ----')
print(daegu_weekend_seg4_list)
for i in LCLS_list:
  try:
    if abs(gs_daegu_weekend_seg4_corr[["confirmed_y", i+'_y']].corr().iloc[1,0])>=cor_val:
      print('대구 휴일 군집4',circulation_list[i], gs_daegu_weekend_seg4_corr[["confirmed_y", i+'_y']].corr().iloc[1,0], \
            ' / semi partial : ', pg.partial_corr(data=gs_daegu_weekend_seg4_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r'], end=' ')
      if abs(pg.partial_corr(data=gs_daegu_weekend_seg4_corr, x='confirmed_y', y=i+'_y', x_covar=i+'_x').loc['pearson','r']) < cor_val:
        print('코로나와 상관없음. 우연에 의함')        
      else:
        print(' 코로나와 상관있음')
        gs_daegu_weekend_seg4_corr40_list.append(circulation_list[i])
  except:
    pass       
print()


대구 편의점 휴일
---- 1군집 (상업지역) ----
['두산동', '범어1동', '성내1동', '중동', '범어2동', '성내2동', '삼덕동']
대구 휴일 군집1 식사 -0.40477000995234147  / semi partial :  -0.33090487529723306 코로나와 상관없음. 우연에 의함
대구 휴일 군집1 간식 -0.4122354285132931  / semi partial :  -0.4115902321839217  코로나와 상관있음
대구 휴일 군집1 마실거리 -0.6089637998368739  / semi partial :  -0.598154403130402  코로나와 상관있음
대구 휴일 군집1 홈&리빙 -0.4731967634206522  / semi partial :  -0.4631221162542667  코로나와 상관있음
대구 휴일 군집1 헬스&뷰티 -0.4544474370130207  / semi partial :  -0.4545788376771052  코로나와 상관있음
대구 휴일 군집1 임신/육아_비중 -0.3688389531025476  / semi partial :  -0.4026773067170738  코로나와 상관있음
---- 2군집 (주거상권) ----
['고산3동', '고산1동', '남산4동', '만촌3동', '범어3동', '범물1동', '범어4동']
대구 휴일 군집2 식사 -0.3663615167102473  / semi partial :  -0.3677916072806988  코로나와 상관있음
대구 휴일 군집2 마실거리 -0.5628328745300459  / semi partial :  -0.429944572116922  코로나와 상관있음
대구 휴일 군집2 사회활동_비중 -0.35624547386780825  / semi partial :  -0.3147383715530945 코로나와 상관없음. 우연에 의함
대구 휴일 군집2 임신/육아_비중 -0.41937225471757367  / semi partial 

## 4-3.군집별 상관관계 유의한 업종/품목 리스트
- 해당 리스트를 'HumanLearning_4. 회귀모델링' 에서 사용

In [ ]:
# 카드 서울, 대구 평일, 휴일 seg 들 list - 16개
print(card_seoul_weekday_seg1_corr40_list)
print(card_seoul_weekday_seg2_corr40_list)
print(card_seoul_weekday_seg3_corr40_list)
print(card_seoul_weekday_seg4_corr40_list)

print(card_seoul_weekend_seg1_corr40_list)
print(card_seoul_weekend_seg2_corr40_list)
print(card_seoul_weekend_seg3_corr40_list)
print(card_seoul_weekend_seg4_corr40_list)

print(card_daegu_weekday_seg1_corr40_list)
print(card_daegu_weekday_seg2_corr40_list)
print(card_daegu_weekday_seg3_corr40_list)
print(card_daegu_weekday_seg4_corr40_list)

print(card_daegu_weekend_seg1_corr40_list)
print(card_daegu_weekend_seg2_corr40_list)
print(card_daegu_weekend_seg3_corr40_list)
print(card_daegu_weekend_seg4_corr40_list)

['레저용품(20)', '의복(42)', '보건위생(71)', '요식업소(80)']
['요식업소(80)']
[]
['요식업소(80)']
['레저용품(20)', '레저업소(21)', '유통업(40)', '의복(42)', '서적문구(50)', '요식업소(80)']
['숙박(10)', '전기(31)', '유통업(40)', '보건위생(71)', '요식업소(80)']
['요식업소(80)']
['보건위생(71)']
['레저업소(21)', '연료판매(33)', '유통업(40)', '사무통신(52)', '의료기관(70)', '보건위생(71)', '요식업소(80)']
['레저업소(21)', '의료기관(70)', '요식업소(80)']
['레저업소(21)', '요식업소(80)']
['레저업소(21)', '요식업소(80)']
['문화취미(22)', '연료판매(33)', '유통업(40)', '서적문구(50)', '사무통신(52)', '요식업소(80)']
['보건위생(71)', '요식업소(80)']
['레저업소(21)', '사무통신(52)', '요식업소(80)']
['레저업소(21)', '연료판매(33)', '의복(42)', '요식업소(80)']


In [ ]:
# 편의점 서울, 대구 평일, 휴일 seg 들 list - 16개
print(gs_seoul_weekday_seg1_corr40_list)
print(gs_seoul_weekday_seg2_corr40_list)
print(gs_seoul_weekday_seg3_corr40_list)
print(gs_seoul_weekday_seg4_corr40_list)

print(gs_seoul_weekend_seg1_corr40_list)
print(gs_seoul_weekend_seg2_corr40_list)
print(gs_seoul_weekend_seg3_corr40_list)
print(gs_seoul_weekend_seg4_corr40_list)

print(gs_daegu_weekday_seg1_corr40_list)
print(gs_daegu_weekday_seg2_corr40_list)
print(gs_daegu_weekday_seg3_corr40_list)
print(gs_daegu_weekday_seg4_corr40_list)

print(gs_daegu_weekend_seg1_corr40_list)
print(gs_daegu_weekend_seg2_corr40_list)
print(gs_daegu_weekend_seg3_corr40_list)
print(gs_daegu_weekend_seg4_corr40_list)

['마실거리']
['식사']
[]
[]
['취미&여가활동_비중', '임신/육아_비중']
['식사', '임신/육아_비중']
[]
['마실거리', '임신/육아_비중']
['식사', '마실거리', '홈&리빙']
['마실거리']
[]
['마실거리']
['간식', '마실거리', '홈&리빙', '헬스&뷰티', '임신/육아_비중']
['식사', '마실거리', '임신/육아_비중']
[]
['마실거리']


## 4-4. 상관 증가했다가 세미 감소한 업종 시각화 (우연성제거)

In [ ]:
# 대구 평일 군집3 주거지역 마실거리 

import plotly.offline as pyo
import plotly.graph_objects as go

from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.update_layout(
    title='대구 평일 주거지역 마실거리',
    title_x = 0.5
)

gs_daegu_weekday_seg3_corr['STD_DD'] = pd.to_datetime(gs_daegu_weekday_seg3_corr['STD_DD'], format='%Y%m%d', errors='raise')

trace2 = go.Scatter(x = gs_daegu_weekday_seg3_corr['STD_DD'], y = gs_daegu_weekday_seg3_corr['confirmed_x'], mode = 'lines', name = '코로나')
trace3 = go.Scatter(x = gs_daegu_weekday_seg3_corr['STD_DD'], y = gs_daegu_weekday_seg3_corr['mul_LCLS_30_P_y'], mode = 'lines', name = '마실거리_20')
trace4 = go.Scatter(x = gs_daegu_weekday_seg3_corr['STD_DD'], y = gs_daegu_weekday_seg3_corr['mul_LCLS_30_P_x'], mode = 'lines', name = '마실거리_19')

fig.add_trace(trace2, secondary_y=False)
fig.add_trace(trace3, secondary_y=True)
fig.add_trace(trace4, secondary_y=True)

fig.update_yaxes(title_text="<b>코로나</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>마실거리</b>", secondary_y=True)

pyo.iplot(fig)

In [ ]:
# 대구 평일 군집3 주거지역 마실거리 rolling
import plotly.offline as pyo
import plotly.graph_objects as go

gs_daegu_weekday_seg3_corr_rolling3 = gs_daegu_weekday_seg3_corr[['confirmed_y','mul_LCLS_30_P_y', 'mul_LCLS_30_P_x']].rolling(window=7).mean()

from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.update_layout(
#    title='대구 평일 주거지역 마실거리',
#    title_x = 0.5,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), ## 범례 그래프위에
)

gs_daegu_weekday_seg3_corr['STD_DD'] = pd.to_datetime(gs_daegu_weekday_seg3_corr['STD_DD'], format='%Y%m%d', errors='raise')

trace2 = go.Scatter(x = gs_daegu_weekday_seg3_corr['STD_DD'], y = gs_daegu_weekday_seg3_corr_rolling3['confirmed_y'], mode = 'lines', name = '신규 코로나 확진자 수')
trace3 = go.Scatter(x = gs_daegu_weekday_seg3_corr['STD_DD'], y = gs_daegu_weekday_seg3_corr_rolling3['mul_LCLS_30_P_y'], mode = 'lines', name = '20 마실거리 매출액')
trace4 = go.Scatter(x = gs_daegu_weekday_seg3_corr['STD_DD'], y = gs_daegu_weekday_seg3_corr_rolling3['mul_LCLS_30_P_x'], mode = 'lines', name = '19 마실거리 매출액')
              

fig.add_trace(trace2, secondary_y=False)
fig.add_trace(trace3, secondary_y=True)
fig.add_trace(trace4, secondary_y=True)

fig.update_yaxes(title_text="<b>신규 코로나 확진자 수</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>매출액</b>", secondary_y=True)

pyo.iplot(fig)